<a href="https://colab.research.google.com/github/luisewoehlke/Honours-Project/blob/master/jupyterNotebooks/PDIoT_lab3_social_signals_no_gyro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 3 - Activity Recognition with Machine Learning**

This notebook implements a machine learning workflow to recognize different physical activities from Respeck sensor data. The dataset includes multiple 30-second recordings of various physical activities (e.g., ascending stairs, shuffle walking, sitting-standing) stored in separate CSV files for each activity.

You will then use the model you develop here and deploy it inside your Android app for live classification.

In this week, you will not have access to the full dataset as of yet. However, you can complete this lab by combining the data that you and your group mates have collected in Coursework 1 as proof-of-concept first for when you eventually receive the full dataset.


# Imports

In [2]:
import sklearn

In [3]:
# Importing libraries that will be used
import pandas as pd
import numpy as np
import glob
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report

# Reading Files
Reading files from your dataset

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Put in the path of your dataset here
your_dataset_path = "C:/Users/luise/OneDrive - University of Edinburgh/uni/pdiot/cw3 data/Respeck/Respiratory/"
your_dataset_path = "C:/Users/luise/OneDrive - University of Edinburgh/uni/pdiot/cw3 data/Respeck all/Respiratory/"
your_dataset_path = "/content/drive/My Drive/Colab Notebooks/cw3 data/Respeck all/Respiratory/"

This line uses the glob module to find all file paths that match a specified pattern. The 'glob.glob()' function returns a list of file paths that match the given pattern. `your_dataset_path` should be the directory where your dataset files are located.

The `*` is a wildcard character that matches any string of characters,  so this pattern retrieves all folders in the 'your_dataset_path' directory.

Below is just an example of what your dataset folder can look like. You should refer to the Coursework 3 instructions on what classes your model(s) are expected to be able to classify. Within your dataset directory, there should be subfolders, each representing a class of activity.

In [6]:
glob.glob(your_dataset_path + "*")

['/content/drive/My Drive/Colab Notebooks/cw3 data/Respeck all/Respiratory/breathingNormally',
 '/content/drive/My Drive/Colab Notebooks/cw3 data/Respeck all/Respiratory/hyperventilation',
 '/content/drive/My Drive/Colab Notebooks/cw3 data/Respeck all/Respiratory/other',
 '/content/drive/My Drive/Colab Notebooks/cw3 data/Respeck all/Respiratory/coughing']

To see the files in each subfolder you can similarly do:

In [7]:
activity_folder = "other"
glob.glob(your_dataset_path + "/"+activity_folder+"/*")

['/content/drive/My Drive/Colab Notebooks/cw3 data/Respeck all/Respiratory//other/s77_respeck_lyingRight_talking.csv',
 '/content/drive/My Drive/Colab Notebooks/cw3 data/Respeck all/Respiratory//other/s77_respeck_sitting_eating.csv',
 '/content/drive/My Drive/Colab Notebooks/cw3 data/Respeck all/Respiratory//other/s77_respeck_lyingLeft_laughing.csv',
 '/content/drive/My Drive/Colab Notebooks/cw3 data/Respeck all/Respiratory//other/s77_respeck_lyingStomach_talking.csv',
 '/content/drive/My Drive/Colab Notebooks/cw3 data/Respeck all/Respiratory//other/s77_respeck_lyingStomach_singing.csv',
 '/content/drive/My Drive/Colab Notebooks/cw3 data/Respeck all/Respiratory//other/s77_respeck_lyingLeft_singing.csv',
 '/content/drive/My Drive/Colab Notebooks/cw3 data/Respeck all/Respiratory//other/s77_respeck_lyingBack_talking.csv',
 '/content/drive/My Drive/Colab Notebooks/cw3 data/Respeck all/Respiratory//other/s77_respeck_standing_singing.csv',
 '/content/drive/My Drive/Colab Notebooks/cw3 data/R

# Functions

## Load list of files in an activity folder

In [8]:
def load_files_from_folder(folder_path):
    """
    Load all CSV files from a folder and return a list of file paths.

    Parameters:
    folder_path (str): The path to the folder containing CSV files.

    Returns:
    list: A list of file paths for all CSV files in the folder.
    """

    # Initialize an empty list to store the full file paths of the CSV files
    file_paths = []

    # Loop through all the files in the given folder
    for file_name in os.listdir(folder_path):
        # Check if the file has a .csv extension (ignores other files)
        if file_name.endswith('.csv'):
            # Construct the full file path by joining the folder path and the file name
            full_file_path = os.path.join(folder_path, file_name)

            # Append the full file path to the file_paths list
            file_paths.append(full_file_path)

    # Return the complete list of CSV file paths
    return file_paths

## Train and test set split from list of files

In [9]:
def split_files(file_list, test_size=0.2):
    """
    Split the list of files into training and test sets.

    Parameters:
    file_list (list): List of file paths to be split into train and test sets.
    test_size (float): The proportion of files to allocate to the test set.
                       Default is 0.2, meaning 20% of the files will be used for testing.

    Returns:
    tuple:
        - train_files (list): List of file paths for the training set.
        - test_files (list): List of file paths for the test set.
    """

    # Split the file list into training and test sets using train_test_split from scikit-learn
    # test_size defines the proportion of the data to use as the test set (default is 20%)
    # shuffle=True ensures that the files are shuffled randomly before splitting
    train_files, test_files = train_test_split(file_list, test_size=test_size, shuffle=True)

    # Return the train and test file lists
    return train_files, test_files

## Sliding Window

In time series Activity Recognition, a sliding window is a commonly used technique to segment continuous sensor data (such as accelerometer readings) into smaller, fixed-length overlapping or non-overlapping time intervals, or windows. Each window contains a sequence of sensor measurements that represent a short period of time, and this segmented data is used to extract features or make predictions about the activity happening within that window.

### Key Concepts of a Sliding Window
1.   **Window Size:** This refers to the length of each segment or window, typically defined in terms of the number of time steps or the duration (e.g., 2 seconds). The window size should be chosen carefully to capture enough information about the activity without making the window too large.
2.   **Step Size:** The step size determines how far the window moves forward after each step. If the step size is smaller than the window size, the windows will overlap. For example, if the window size is 5 seconds and the step size is 2 seconds, there will be a 3-second overlap between consecutive windows. Overlapping windows provide more data for analysis and can help smooth out predictions by capturing transitional activities.
3.   **Non-Overlapping Windows:** If the step size is equal to the window size, the windows do not overlap. This method provides distinct segments of data but may miss transitional phases between activities.

### Why Sliding Windows for Activity Recognition?

* Segmentation of Continuous Data: Activity recognition systems work with continuous streams of sensor data, and the sliding window helps segment these into manageable pieces to classify activities within specific intervals.

* Context Capturing: Human activities are often complex and spread across time. By using a sliding window, you can capture context across a short duration, which may include transitions or small fluctuations in the activity (e.g., a person moving from sitting to standing).

* Feature Extraction: Within each window, features such as mean, variance, frequency domain features, etc., can be extracted to help classify the activity.

* Real-Time Recognition: In real-time systems, the sliding window allows for continuous monitoring and updating of predictions as new data arrives.



In [10]:
def load_and_apply_sliding_windows(file_paths, window_size, step_size, label):
    """
    Load the data from each file, apply sliding windows, and return the windows and labels.

    Parameters:
    file_paths (list): List of file paths to CSV files. Each file contains sensor data (e.g., accelerometer, gyroscope).
    window_size (int): The size of each sliding window (number of time steps).
    step_size (int): The step size (stride) between consecutive windows.
    label (int or str): The label for the activity corresponding to the folder.
                        This label will be assigned to each sliding window extracted from the data.

    Returns:
    tuple:
        - windows (numpy.ndarray): A 3D array of sliding windows, where each window has the shape
                                   (num_windows, window_size, num_features).
        - labels (numpy.ndarray): A 1D array of labels, where each label corresponds to a sliding window.
    """
    # Initialize lists to store sliding windows and their corresponding labels
    windows = []
    labels = []

    # Loop through each file in the provided file paths
    for file_path in file_paths:
        # Load the CSV file into a pandas DataFrame
        data = pd.read_csv(file_path)

        # Select the columns containing the necessary sensor data (acceleration and gyroscope readings)
        # These columns might vary depending on your dataset's structure
        data = data[['accel_x', 'accel_y', 'accel_z']]

        # Convert the DataFrame into a numpy array for faster processing in the sliding window operation
        data = data.to_numpy()

        # Get the number of samples (rows) and features (columns) in the data
        num_samples, num_features = data.shape

        # Apply sliding windows to the data
        # The range function defines the start of each window, moving step_size increments at a time
        for i in range(0, num_samples - window_size + 1, step_size):
            # Extract a window of size 'window_size' from the current position 'i'
            window = data[i:i + window_size, :]

            # Append the window to the windows list
            windows.append(window)

            # Assign the activity label to the window and append it to the labels list
            labels.append(label)

    # Convert the lists of windows and labels into numpy arrays for efficient numerical operations
    return np.array(windows), np.array(labels)

## Load and Split Train Test for Each Activity Folder

This function processes the sensor data for a specific activity, such as 'walking' or 'running', stored in its respective folder. It splits the data into training and testing sets, applies sliding windows, and labels the windows with the corresponding activity. This function can be used repeatedly for each activity to process and prepare data for training and evaluation.

In [11]:
def process_activity(activity, label, dataset_path, window_size=50, step_size=50, test_size=0.2):
    """
    Processes an activity folder by loading the file list, splitting them into
    train and test sets, and applying sliding windows to the files.

    Args:
        activity (str): Name of the activity (folder name). This refers to the specific physical activity
                        like 'walking', 'running', etc.
        label (int): Numeric label corresponding to the activity, used for classification.
        dataset_path (str): Base path where the activity folders are located.
        window_size (int): Size of the sliding window, i.e., the number of time steps included in each window.
                           Default is 50.
        step_size (int): Step size for the sliding window, i.e., how far the window moves along the data.
                         Default is 50 (no overlap between windows).
        test_size (float): Proportion of files to use for testing. Default is 0.2, meaning 20% of files will
                           be allocated to the test set.

    Returns:
        tuple:
            - train_windows (numpy.ndarray): Sliding windows from the training files.
            - train_labels (numpy.ndarray): Corresponding labels for the training windows.
            - test_windows (numpy.ndarray): Sliding windows from the test files.
            - test_labels (numpy.ndarray): Corresponding labels for the test windows.
    """
    # Construct the full folder path where the activity files are stored
    folder_path = os.path.join(dataset_path, activity)

    # Load all CSV file paths for the given activity from the folder
    file_list = load_files_from_folder(folder_path)

    # Split the file list into training and testing sets
    # train_files: files used for training
    # test_files: files used for testing
    train_files, test_files = split_files(file_list, test_size=test_size)

    # Apply sliding windows to the training files
    # The function 'load_and_apply_sliding_windows' returns the sliding windows (segments) and their corresponding labels
    train_windows, train_labels = load_and_apply_sliding_windows(train_files, window_size, step_size, label)

    # Apply sliding windows to the testing files
    test_windows, test_labels = load_and_apply_sliding_windows(test_files, window_size, step_size, label)

    # Return the sliding windows and their labels for both training and testing sets
    return train_windows, train_labels, test_windows, test_labels

## Combine Data
The function combines the sliding window data and their corresponding labels from multiple activities (e.g., walking, running, etc.) into single arrays.

In [12]:
def combine_data(train_test_data, data_type):
    """
    Combines the sliding windows and labels from all activities into a single
    array for either training or testing.

    Args:
        train_test_data (dict): Dictionary containing the sliding window data for all activities.
                                Each key in the dictionary corresponds to an activity, and the value is another
                                dictionary with the keys 'train_windows', 'train_labels', 'test_windows', 'test_labels'.
        data_type (str): Either 'train' or 'test' to specify which data to combine (e.g., 'train_windows' or 'test_windows').

    Returns:
        tuple:
            - windows (numpy.ndarray): Concatenated windows from all activities for either training or testing.
            - labels (numpy.ndarray): Concatenated labels corresponding to the windows from all activities.
    """

    # Extract the list of sliding windows for the specified data type (either 'train' or 'test') from each activity
    # For example, if data_type is 'train', it extracts 'train_windows' for all activities
    windows_list = [train_test_data[activity][f'{data_type}_windows'] for activity in train_test_data]

    # Similarly, extract the list of labels corresponding to the windows for each activity
    labels_list = [train_test_data[activity][f'{data_type}_labels'] for activity in train_test_data]

    # Concatenate all the sliding windows into a single numpy array along the first axis (rows)
    # This creates one large array of windows from all the activities combined
    concatenated_windows = np.concatenate(windows_list, axis=0)

    # Concatenate all the labels into a single numpy array along the first axis (rows)
    # The labels are now aligned with the concatenated windows
    concatenated_labels = np.concatenate(labels_list, axis=0)

    # Return the concatenated windows and labels as a tuple
    return concatenated_windows, concatenated_labels

## 1D CNN Model

This function, `build_1d_cnn_model`, creates and compiles a 1D Convolutional Neural Network (CNN) for multi-class classification tasks.

### Function Overview

Input Parameters
* `input_shape`: Specifies the shape of the input data. It represents (timesteps, features), where timesteps refer to the length of the time series (e.g., 50 windows), and features represent the number of measurements in each time step (e.g., accelerometer readings).
* `num_classes`: The number of output classes for the classification problem. For example, if you're classifying six different activities, num_classes would be 6.

Returns
* The function returns a compiled 1D CNN model that is ready to be trained on your data.

<hr>

### Function Breakdown
1.   Model Initialization:
    * `model = Sequential()`: Initializes a Sequential model, which means layers will be stacked on top of each other in a linear fashion.
2.   First Convolutional Layer
    * `Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape)`
        * This is the first 1D convolutional layer
        * `filters=64`: The layer applies 64 filters (or kernels) over the input data.
        * `kernel_size=3`: Each filter will cover 3 timesteps at a time (a window of 3).
        * `activation='relu'`: The Rectified Linear Unit (ReLU) activation function introduces non-linearity and helps the model learn complex patterns.
        * `input_shape=input_shape`: Specifies the shape of the input data.
    * `MaxPooling1D(pool_size=2)`: This pooling layer reduces the dimensionality of the data by taking the maximum value from each 2-timestep window (`pool_size=2`). This helps reduce computational complexity and captures the most important features.
3. Second Convolutional Layer:
    * `Conv1D(filters=128, kernel_size=3, activation='relu')`
        * This is the second convolutional layer, similar to the first, but with 128 filters, which allow the network to learn more complex features from the data.
        * `kernel_size=3` and activation='relu' function in the same way as the first Conv1D layer.
    * `MaxPooling1D(pool_size=2)`: Another pooling layer to downsample the output, further reducing the data’s dimensionality.
4. Flattening Layer:
    * `Flattening`: Converts the 2D output of the convolutional and pooling layers into a 1D vector. This is necessary because the next layer is fully connected, and it requires a 1D input.
5. Fully Connected Layer:
    * `Dense(128, activation='relu')`: This is a fully connected layer with 128 units/neurons. Each neuron is connected to every input from the flattened output. The ReLU activation function is used again to introduce non-linearity and help the model learn complex relationships.
6. Dropout Layer:
    * `Dropout(0.5)`: This layer randomly sets 50% of the neurons to zero during training to prevent overfitting. It helps the model generalize better to unseen data.
7. Output Layer:
    * `Dense(num_classes, activation='softmax')`: This is the output layer with num_classes neurons, one for each class in the classification problem. The softmax activation function ensures the output values represent probabilities that sum to 1, useful for multi-class classification.
8. Compiling the model
    * model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']):
        * Optimizer: 'adam': Adam is an optimization algorithm that adjusts the learning rate during training to improve performance.
        * Loss: 'categorical_crossentropy': This loss function is used for multi-class classification problems where the target variable is one-hot encoded (i.e., represented as a vector of 0s and 1s).
        * Metrics: ['accuracy']: The accuracy metric is used to evaluate the model’s performance during training and testing.


In [13]:
def build_1d_cnn_model(input_shape, num_classes):
    """
    Builds and compiles a 1D CNN model for multi-class classification.

    Args:
        input_shape (tuple): The shape of the input data (timesteps, features).
        num_classes (int): The number of output classes.

    Returns:
        model (Sequential): Compiled 1D CNN model.
    """
    model = Sequential()

    # First Conv1D layer
    # You can try experimenting with different filters, kernel_size values and activiation functions
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))

    # Second Conv1D layer
    # You can try experimenting with different filters, kernel_size values and activiation functions
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))

    # Flatten the output from the convolutional layers
    model.add(Flatten())

    # Fully connected layer
    model.add(Dense(128, activation='relu'))

    # Dropout layer for regularization
    # You can try experimenting with different dropout rates
    model.add(Dropout(0.5))

    # Output layer with softmax for multi-class classification
    model.add(Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    #  Prints a detailed summary of the model, showing the layers, their output shapes, and the number of trainable parameters
    model.summary()

    return model

# Classification Pipeline

## Step 1: Prepare and Preprocess the Data

In [14]:
# Define activity folders and corresponding labels
# Each key is the name of the physical activity, and the corresponding value is the numeric label
# These labels will be used as the target variable for classification.
activities = {
    'breathingNormally': 0,
    'coughing': 1,
    'hyperventilation': 2,
    'other': 3
}

In [15]:
# Dictionary to store sliding windows and labels for both train and test sets for each activity
# This will hold the training and test data after processing each activity.
train_test_data = {}

# Loop through each activity folder and process the data
# Note, if you have large amounts of data, this step may take a while
for activity, label in activities.items():
    # Initialize an empty dictionary for each activity to store train and test windows and labels
    train_test_data[activity] = {}

    # Call process_activity() to process the data for the current activity folder
    # It loads the data, applies sliding windows, splits it into train and test sets,
    # and returns the respective sliding windows and labels for both sets.
    (train_test_data[activity]['train_windows'], train_test_data[activity]['train_labels'],
     train_test_data[activity]['test_windows'], train_test_data[activity]['test_labels']) = process_activity(
        activity, label, your_dataset_path, window_size=50, step_size=50)

# Explanation:
    # - 'train_windows' and 'train_labels' store the windows and labels from the training files.
    # - 'test_windows' and 'test_labels' store the windows and labels from the test files.
    # - `your_dataset_path` should be replaced with the actual path to your dataset.
    # - `process_activity` handles all the steps of loading data, splitting it, and applying sliding windows.

Now that each activity has been processed and stored in train_test_data, we need to combine the sliding windows and labels from all activities into unified arrays (one for training and one for testing) for model training.

In [16]:
# Combine the sliding windows and labels for the training data from all activities
# The combine_data() function concatenates the windows and labels across activities
X_train, y_train = combine_data(train_test_data, 'train')

# Combine the sliding windows and labels for the test data from all activities
X_test, y_test = combine_data(train_test_data, 'test')

# Explanation:
# - `combine_data()` takes in the `train_test_data` dictionary and the data type ('train' or 'test') to specify
#   whether we are combining training or testing data.
# - It retrieves and concatenates the windows and labels from all activities into single arrays
#   (`X_train` and `y_train` for training, `X_test` and `y_test` for testing).
# - `X_train` and `X_test` are 3D arrays of sliding windows (shape: num_windows, window_size, num_features).
# - `y_train` and `y_test` are 1D arrays containing the activity labels corresponding to each window.

### One-Hot Encode Labels (for multi-class classification)
If you have more than two classes, you'll need to one-hot encode the labels, especially if your model will use categorical cross-entropy loss.

One-Hot Encoding converts categorical labels into binary vectors (one-hot encoded format). Each class label is represented as a binary vector with 1 for the correct class and 0 for others. This is necessary for training models that use categorical_crossentropy as the loss function, such as a neural network.

In [17]:
# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Reshape y_train to a 2D array to meet the input format requirements of OneHotEncoder
# - y_train is originally a 1D array of labels (shape: [num_samples]), but OneHotEncoder expects a 2D array of shape (num_samples, 1).
# - reshape(-1, 1): The -1 means 'infer the correct size based on the other dimensions' (i.e., it adapts based on the length of y_train).
# OneHotEncoder will then create a binary vector for each label.
y_train_one_hot = encoder.fit_transform(y_train.reshape(-1, 1))

# Apply the same transformation to the test labels (y_test)
# - Since the encoder is already fitted on the training data, we use transform() for the test set.
# - Reshape y_test to (num_samples, 1) for compatibility with the encoder.
y_test_one_hot = encoder.transform(y_test.reshape(-1, 1))

# Explanation:
# - y_train_one_hot and y_test_one_hot are now 2D arrays where each row is a one-hot encoded binary vector corresponding to a class label.
# - The number of columns in the one-hot encoded labels equals the number of unique classes (activities).
# For example, if there are 6 unique activities, the encoded vector will have 6 elements, with a '1' indicating the correct class.

In [18]:
# Print the shapes of the training and test arrays to verify that everything has been combined correctly
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")
# Print the shapes of the one-hot encoded labels to verify that the transformation was successful
print(f"y_train_one_hot shape: {y_train_one_hot.shape}, y_test_one_hot shape: {y_test_one_hot.shape}")

# Explanation of shapes:
# - The shape of y_train_one_hot will be (num_samples, num_classes), where:
#     - num_samples is the number of training windows.
#     - num_classes is the number of unique activities (the length of the one-hot vectors).
# - Similarly, y_test_one_hot will have the same number of columns (num_classes) as y_train_one_hot but will have fewer rows (corresponding to the number of test windows).

X_train shape: (57230, 50, 3), y_train shape: (57230,)
X_test shape: (14337, 50, 3), y_test shape: (14337,)
y_train_one_hot shape: (57230, 4), y_test_one_hot shape: (14337, 4)


## Step 2: Build the 1D-CNN Model
Call our `build_1d_cnn_model` functionto build our model

In [46]:
# Determine the input shape for the model
input_shape = (X_train.shape[1], X_train.shape[2])

# Determine the number of output classes (num_classes)
num_classes = y_train_one_hot.shape[1]

# Build and compile the model
# The function will return a compiled model ready for training
model = build_1d_cnn_model(input_shape, num_classes)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_16 (Conv1D)                   │ (None, 48, 64)              │             640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_16 (MaxPooling1D)      │ (None, 24, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_17 (Conv1D)                   │ (None, 22, 128)             │          24,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_17 (MaxPooling1D)      │ (None, 11, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_8 (Flatten)                  │ (None, 1408)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 128)                 │         180,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_22 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 4)                   │             516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 206,212 (805.52 KB)

 Trainable params: 206,212 (805.52 KB)

 Non-trainable params: 0 (0.00 B)

## Step 3: Train the CNN Model

Train the 1D CNN model using the training data and validate on the test data. The model will learn to map input sliding windows to their corresponding activity labels.

`model.fit()` is used to train the neural network model. It takes several parameters:
* `X_train`: The input training data (sliding windows), with shape (num_samples, window_size, num_features).
* `y_train_one_hot`: The corresponding one-hot encoded labels for the training data, with shape (num_samples, num_classes).
* `epochs`: Number of times the entire training dataset is passed through the model. You can try adjusting the number of epochs and compare the difference in model performance. In this case, we are training for 20 epochs, meaning the model will see the entire training set 20 times.
* `batch_size`: Number of samples processed before the model's weights are updated. Here, the batch size is set to 32, meaning the model will process 32 samples at a time before updating its parameters.
* `validation_data`: This parameter allows us to evaluate the model's performance on the test data after each epoch.
*`(X_test, y_test_one_hot)`: These are the input test data and corresponding one-hot encoded test labels.

# Luise's accuracy improvement trials:

## ReduceLROnPlateau

In [47]:
# 1. ReduceLROnPlateau // reduce learning rate when val_accuracy stops improving
from tensorflow.keras.callbacks import ReduceLROnPlateau
lr_scheduler = ReduceLROnPlateau(
    monitor='val_accuracy',   # Track validation accuracy for learning rate adjustments
    factor=0.5,               # Reduce learning rate by half when triggered
    patience=3,               # Wait for 3 epochs of no improvement before reducing
    min_lr=1e-5               # Set a minimum learning rate to prevent it from going too low
)
history = model.fit(X_train, y_train_one_hot,
                    epochs=20,         # Train the model for 20 epochs
                    batch_size=32,     # Use a batch size of 32
                    validation_data=(X_test, y_test_one_hot),   # Validate on the test set after each epoch
                    callbacks=[lr_scheduler])

Epoch 1/20
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.5226 - loss: 1.1957 - val_accuracy: 0.5899 - val_loss: 0.9636 - learning_rate: 0.0010
Epoch 2/20
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.5561 - loss: 0.9992 - val_accuracy: 0.5825 - val_loss: 0.9419 - learning_rate: 0.0010
Epoch 3/20
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.5641 - loss: 0.9549 - val_accuracy: 0.5892 - val_loss: 0.9153 - learning_rate: 0.0010
Epoch 4/20
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.5713 - loss: 0.9247 - val_accuracy: 0.5985 - val_loss: 0.8884 - learning_rate: 0.0010
Epoch 5/20
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.5886 - loss: 0.8891 - val_accuracy: 0.6133 - val_loss: 0.8617 - learning_rate: 0.0010
Epoch 6/20
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.6052 - loss: 0.8575 - val_accuracy: 0.6295 - val_loss: 0.8096 - learning_rate: 0.0010
Epoch 7/20
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.6189 - 

### test

In [51]:
from sklearn.metrics import recall_score, classification_report
import numpy as np

def test(model, X_test, y_test_one_hot):
  # Get predicted probabilities for the test set
  y_pred_probs = model.predict(X_test)

  # Convert the predicted probabilities to class labels (taking the argmax of the probabilities)
  y_pred_classes = np.argmax(y_pred_probs, axis=1)

  # Convert the true test labels from one-hot encoding back to class labels
  y_true_classes = np.argmax(y_test_one_hot, axis=1)

  # Generate the classification report
  report = classification_report(y_true_classes, y_pred_classes, digits=4)

  # Print the classification report
  print(report)

  # Calculate and print per-class recall
  classes = np.unique(y_true_classes)

  print("Per-class Recall:")
  for cls in classes:
      # Calculate recall for the current class
      recall = recall_score(y_true_classes, y_pred_classes, labels=[cls], average=None)[0]
      print(f"Class {cls}: {recall:.4f}")

  # Calculate accuracy for each class
  for cls in classes:
      # Get indices for the current class
      cls_indices = (y_true_classes == cls)

      # Include both true positives (correctly predicted for this class) and true negatives (correctly not predicted as this class)
      cls_accuracy = np.mean((y_pred_classes == cls) == (y_true_classes == cls))

      # Store the accuracy in the dictionary
      per_class_accuracy[cls] = cls_accuracy

  # Print the per-class accuracy
  print("Per-class Accuracy:")
  for cls, acc in per_class_accuracy.items():
      print(f"Class {cls}: {acc:.4f}")

test(model,X_test,y_test_one_hot)

449/449 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
              precision    recall  f1-score   support

           0     0.7070    0.5276    0.6042      2794
           1     0.7234    0.6579    0.6891      2163
           2     0.6370    0.2483    0.3573      2163
           3     0.6447    0.8434    0.7308      7217

    accuracy                         0.6641     14337
   macro avg     0.6780    0.5693    0.5953     14337
weighted avg     0.6675    0.6641    0.6435     14337

Per-class Recall:
Class 0: 0.5276
Class 1: 0.6579
Class 2: 0.2483
Class 3: 0.8434
Per-class Accuracy:
Class 0: 0.8653
Class 1: 0.9104
Class 2: 0.8652
Class 3: 0.6872


## 2. automatic hyperparameter tuner:

In [20]:
pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.7 MB/s eta 0:00:00


In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, LSTM
from tensorflow.keras.regularizers import l2
import keras_tuner as kt

In [23]:
def build_tuning_model(hp):
    model = Sequential()

    # First Conv1D Layer
    model.add(Conv1D(
        filters=hp.Choice('filters_1', values=[64, 128, 256]),
        kernel_size=hp.Choice('kernel_size_1', values=[3, 5, 7]),
        activation='relu',
        input_shape=input_shape
    ))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(rate=hp.Choice('dropout_1', values=[0.3, 0.5, 0.6])))

    # Second Conv1D Layer
    model.add(Conv1D(
        filters=hp.Choice('filters_2', values=[64, 128, 256]),
        kernel_size=hp.Choice('kernel_size_2', values=[3, 5, 7]),
        activation='relu',
        kernel_regularizer=l2(hp.Choice('l2_regularization', values=[1e-4, 1e-3]))
    ))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(rate=hp.Choice('dropout_2', values=[0.3, 0.5, 0.6])))

    # Optional LSTM Layer
    if hp.Boolean("use_lstm"):
        model.add(LSTM(units=hp.Choice('lstm_units', values=[32, 64, 128]), return_sequences=False))

    # Dense Layers
    model.add(Flatten())

    model.add(Dense(128, activation='relu')) #gpt suggested 64 but original model has 128

    model.add(Dropout(rate=hp.Choice('dropout_3', values=[0.3, 0.5, 0.6])))

    model.add(Dense(num_classes, activation='softmax'))  # Assuming 4 classes for respiratory activities

    # Compile Model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    model.summary()

    return model

In [ ]:
tuner = kt.RandomSearch(
    build_tuning_model,
    objective='val_accuracy',  # Tuning for validation accuracy
    max_trials=20,             # Number of different hyperparameter sets to try
    executions_per_trial=2,    # Average results over multiple runs to reduce variance
    directory='my_tuning_dir', # Directory to save tuning results
    project_name='respiratory_activity_classification_no_gyro'
)

# Split your data if not already done
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

tuner.search(X_train, y_train_one_hot,
             epochs=20,               # Number of epochs for each trial
             validation_data=(X_test, y_test_one_hot),
             callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)])

best_model1 = tuner.get_best_models(num_models=1)[0]

# Evaluate the best model
best_model1.evaluate(X_test, y_test_one_hot)  # Replace with your test data
tuner.results_summary()

Trial 20 Complete [00h 02m 25s]
val_accuracy: 0.5933031141757965

Best val_accuracy So Far: 0.6573770344257355
Total elapsed time: 01h 20m 36s


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 48, 256)             │           2,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 48, 256)             │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 24, 256)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 24, 256)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 20, 128)             │         163,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 20, 128)             │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 10, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 10, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4)                   │             516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 226,308 (884.02 KB)

 Trainable params: 225,540 (881.02 KB)

 Non-trainable params: 768 (3.00 KB)

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 32 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


448/448 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6717 - loss: 0.8577
Results summary
Results in my_tuning_dir/respiratory_activity_classification_no_gyro
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 16 summary
Hyperparameters:
filters_1: 256
kernel_size_1: 3
dropout_1: 0.3
filters_2: 128
kernel_size_2: 5
l2_regularization: 0.001
dropout_2: 0.3
use_lstm: True
dropout_3: 0.3
learning_rate: 0.001
lstm_units: 64
Score: 0.6573770344257355

Trial 11 summary
Hyperparameters:
filters_1: 256
kernel_size_1: 3
dropout_1: 0.3
filters_2: 128
kernel_size_2: 7
l2_regularization: 0.001
dropout_2: 0.5
use_lstm: True
dropout_3: 0.6
learning_rate: 0.001
lstm_units: 32
Score: 0.6270666122436523

Trial 05 summary
Hyperparameters:
filters_1: 256
kernel_size_1: 3
dropout_1: 0.5
filters_2: 128
kernel_size_2: 5
l2_regularization: 0.0001
dropout_2: 0.6
use_lstm: False
dropout_3: 0.6
learning_rate: 0.001
lstm_units: 128
Score: 0.6212417185306549

Trial 00 summary
Hyperparam

### test

## best model previous tuner found on last year's data

In [24]:
# best model tuning found:
def build_best_tuning_model(custom_input_shape=input_shape):
    model = Sequential()

    # First Conv1D Layer
    model.add(Conv1D(
        filters=128,
        kernel_size=5,
        activation='relu',
        input_shape=custom_input_shape
    ))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(rate=0.3))

    # Second Conv1D Layer
    model.add(Conv1D(
        filters=256,
        kernel_size=5,
        activation='relu',
        kernel_regularizer=l2(1e-3)
    ))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(rate=0.3))

    # Optional LSTM Layer
    model.add(LSTM(units=128, return_sequences=False))

    # Dense Layers
    model.add(Flatten())

    model.add(Dense(128, activation='relu')) #gpt suggested 64 but original model has 128

    model.add(Dropout(rate=0.3))

    model.add(Dense(num_classes, activation='softmax'))  # Assuming 4 classes for respiratory activities

    # Compile Model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    model.summary()

    return model

best_model = build_best_tuning_model()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_14 (Conv1D)                   │ (None, 46, 128)             │           2,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_12               │ (None, 46, 128)             │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_14 (MaxPooling1D)      │ (None, 23, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_19 (Dropout)                 │ (None, 23, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_15 (Conv1D)                   │ (None, 19, 256)             │         164,096 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_13               │ (None, 19, 256)             │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_15 (MaxPooling1D)      │ (None, 9, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_20 (Dropout)                 │ (None, 9, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_6 (LSTM)                        │ (None, 128)                 │         197,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_7 (Flatten)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_21 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 4)                   │             516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 381,828 (1.46 MB)

 Trainable params: 381,060 (1.45 MB)

 Non-trainable params: 768 (3.00 KB)

In [25]:
history = best_model.fit(X_train, y_train_one_hot,
                    epochs=20,         # Train the model for 20 epochs
                    batch_size=32,     # Use a batch size of 32
                    validation_data=(X_test, y_test_one_hot)   # Validate on the test set after each epoch
                )

Epoch 1/20
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 18s 9ms/step - accuracy: 0.5083 - loss: 1.3303 - val_accuracy: 0.5676 - val_loss: 1.0695
Epoch 2/20
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.5527 - loss: 1.1055 - val_accuracy: 0.5994 - val_loss: 1.0007
Epoch 3/20
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.5726 - loss: 1.0306 - val_accuracy: 0.6071 - val_loss: 0.9701
Epoch 4/20
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.5997 - loss: 0.9800 - val_accuracy: 0.6376 - val_loss: 0.9152
Epoch 5/20
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.6122 - loss: 0.9618 - val_accuracy: 0.6545 - val_loss: 0.8686
Epoch 6/20
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.6190 - loss: 0.9281 - val_accuracy: 0.6560 - val_loss: 0.8701
Epoch 7/20
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.6342 - loss: 0.8979 - val_accuracy: 0.6383 - val_loss: 0.8648
Epoch 8/20
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.6346 - loss: 0

### test

In [53]:
test(best_model,X_test,y_test_one_hot)

449/449 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
              precision    recall  f1-score   support

           0     0.6626    0.6016    0.6307      2794
           1     0.8304    0.5479    0.6602      2163
           2     0.7044    0.4087    0.5173      2163
           3     0.6649    0.8401    0.7423      7217

    accuracy                         0.6845     14337
   macro avg     0.7156    0.5996    0.6376     14337
weighted avg     0.6954    0.6845    0.6742     14337

Per-class Recall:
Class 0: 0.6016
Class 1: 0.5479
Class 2: 0.4087
Class 3: 0.8401
Per-class Accuracy:
Class 0: 0.8627
Class 1: 0.9149
Class 2: 0.8849
Class 3: 0.7064


### 5-fold cross-validation

In [23]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.metrics import classification_report, accuracy_score

# Combine training and testing data
X = np.concatenate((X_train, X_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0)

# Set up 5-fold cross-validation
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

acc_per_fold = []
loss_per_fold = []
class_accuracies = {}  # To store per-class accuracies across folds

fold_no = 1
for train_index, test_index in skf.split(X, y):
    print(f'Training for fold {fold_no} ...')

    X_train_fold, X_test_fold = X[train_index], X[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]

    # One-hot encode labels
    num_classes = 4
    y_train_fold_one_hot = to_categorical(y_train_fold, num_classes)
    y_test_fold_one_hot = to_categorical(y_test_fold, num_classes)

    # Build and compile the model
    model = build_best_tuning_model()

    # Train the model
    history = model.fit(X_train_fold, y_train_fold_one_hot,
                        epochs=20,
                        batch_size=32,
                        validation_data=(X_test_fold, y_test_fold_one_hot))

    # Evaluate the model
    scores = model.evaluate(X_test_fold, y_test_fold_one_hot, verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')

    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Get predictions
    y_pred_fold = model.predict(X_test_fold)
    y_pred_classes = np.argmax(y_pred_fold, axis=1)
    y_true_classes = np.argmax(y_test_fold_one_hot, axis=1)

    # Print classification report
    print(f'Classification Report for fold {fold_no}:')
    print(classification_report(y_true_classes, y_pred_classes))

    # Collect per-class accuracies
    unique_classes = np.unique(y_true_classes)
    for cls in unique_classes:
        cls_indices = np.where(y_true_classes == cls)
        cls_accuracy = accuracy_score(y_true_classes[cls_indices], y_pred_classes[cls_indices])
        if cls not in class_accuracies:
            class_accuracies[cls] = []
        class_accuracies[cls].append(cls_accuracy)

    fold_no += 1

# Print the average scores
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(len(acc_per_fold)):
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)}% (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

# Print average per-class accuracies over all folds
print('Average Per-Class Accuracies over all folds:')
for cls in unique_classes:
    avg_accuracy = np.mean(class_accuracies[cls])
    print(f'Class {cls}: {avg_accuracy * 100:.2f}%')

Training for fold 1 ...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_4 (Conv1D)                    │ (None, 46, 128)             │           2,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 46, 128)             │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_4 (MaxPooling1D)       │ (None, 23, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 23, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_5 (Conv1D)                    │ (None, 19, 256)             │         164,096 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 19, 256)             │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_5 (MaxPooling1D)       │ (None, 9, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 9, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 128)                 │         197,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 4)                   │             516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 381,828 (1.46 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_12 (Conv1D)                   │ (None, 46, 128)             │           2,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_10               │ (None, 46, 128)             │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_12 (MaxPooling1D)      │ (None, 23, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_16 (Dropout)                 │ (None, 23, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_13 (Conv1D)                   │ (None, 19, 256)             │         164,096 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_11               │ (None, 19, 256)             │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_13 (MaxPooling1D)      │ (None, 9, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_17 (Dropout)                 │ (None, 9, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 128)                 │         197,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_6 (Flatten)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_18 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 4)                   │             516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 381,828 (1.46 MB)

 Trainable params: 381,060 (1.45 MB)

 Non-trainable params: 768 (3.00 KB)

Epoch 1/20
1790/1790 ━━━━━━━━━━━━━━━━━━━━ 17s 8ms/step - accuracy: 0.5159 - loss: 1.3114 - val_accuracy: 0.5579 - val_loss: 1.1058
Epoch 2/20
1790/1790 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.5645 - loss: 1.0814 - val_accuracy: 0.5902 - val_loss: 0.9951
Epoch 3/20
1790/1790 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.5891 - loss: 1.0175 - val_accuracy: 0.6010 - val_loss: 1.0052
Epoch 4/20
1790/1790 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.6065 - loss: 0.9836 - val_accuracy: 0.6194 - val_loss: 0.9426
Epoch 5/20
1790/1790 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.6219 - loss: 0.9496 - val_accuracy: 0.6271 - val_loss: 0.9335
Epoch 6/20
1790/1790 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.6196 - loss: 0.9397 - val_accuracy: 0.6375 - val_loss: 0.9103
Epoch 7/20
1790/1790 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.6333 - loss: 0.9078 - val_accuracy: 0.6164 - val_loss: 0.9125
Epoch 8/20
1790/1790 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.6353 - loss: 0

In [ ]:
# custom cell to rebuild modified versions of model
model = build_1d_cnn_model(input_shape, num_classes)

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_10 (Conv1D)              │ (None, 48, 64)         │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_10 (MaxPooling1D) │ (None, 24, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_11 (Conv1D)              │ (None, 22, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_11 (MaxPooling1D) │ (None, 11, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 1408)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 128)            │       180,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 206,788 (807.77 KB)

 Trainable params: 206,788 (807.77 KB)

 Non-trainable params: 0 (0.00 B)

## Step 4: Evaluate the Model
After training, you can evaluate the model on the test set:

In [ ]:
history = model.fit(X_train, y_train_one_hot,
                    epochs=20,         # Train the model for 20 epochs
                    batch_size=32,     # Use a batch size of 32
                    validation_data=(X_test, y_test_one_hot)   # Validate on the test set after each epoch
                )

In [28]:
from sklearn.metrics import classification_report
import numpy as np


# Get predicted probabilities for the test set
y_pred_probs = best_model.predict(X_test)

# Convert the predicted probabilities to class labels (taking the argmax of the probabilities)
y_pred_classes = np.argmax(y_pred_probs, axis=1)

# Convert the true test labels from one-hot encoding back to class labels
y_true_classes = np.argmax(y_test_one_hot, axis=1)

# Generate the classification report
report = classification_report(y_true_classes, y_pred_classes, digits=4)

# Print the classification report
print(report)

# Assuming you already have y_pred_classes and y_true_classes
# Get the unique class labels
classes = np.unique(y_true_classes)

# Initialize a dictionary to store per-class accuracy
per_class_accuracy = {}

# Calculate accuracy for each class
for cls in classes:
    # Get indices for the current class
    cls_indices = (y_true_classes == cls)

    # Include both true positives (correctly predicted for this class) and true negatives (correctly not predicted as this class)
    cls_accuracy = np.mean((y_pred_classes == cls) == (y_true_classes == cls))

    # Store the accuracy in the dictionary
    per_class_accuracy[cls] = cls_accuracy

# Print the per-class accuracy
print("Per-class Accuracy:")
for cls, acc in per_class_accuracy.items():
    print(f"Class {cls}: {acc:.4f}")

449/449 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
              precision    recall  f1-score   support

           0     0.6626    0.6016    0.6307      2794
           1     0.8304    0.5479    0.6602      2163
           2     0.7044    0.4087    0.5173      2163
           3     0.6649    0.8401    0.7423      7217

    accuracy                         0.6845     14337
   macro avg     0.7156    0.5996    0.6376     14337
weighted avg     0.6954    0.6845    0.6742     14337

Per-class Accuracy:
Class 0: 0.8627
Class 1: 0.9149
Class 2: 0.8849
Class 3: 0.7064


As you can see from the model performance results, the classification performance isn't exactly impressive. For Coursework 3, your group should explore and experiment with various models, parameters, and techniques in order to improve your model's performance.

# Exporting your model to TFLite

You can use the TFLiteConverter class provided by TensorFlow to convert your trained model into the TensorFlow Lite format. We export models to TensorFlow Lite (TFLite) for several reasons, primarily because TFLite is designed for deployment on edge devices, such as mobile phones, embedded systems, IoT devices, and microcontrollers, where computational resources and power are limited. This is necessary as you will be running your ML models on your Android devices to perform live classification.

In [ ]:
# Convert the trained Keras model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)  # model is your trained Keras model
tflite_model = converter.convert()

# Save the converted model to a .tflite file
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model successfully exported to model.tflite")

INFO:tensorflow:Assets written to: C:\Users\luise\AppData\Local\Temp\tmppf3rz10g\assets


INFO:tensorflow:Assets written to: C:\Users\luise\AppData\Local\Temp\tmppf3rz10g\assets


Saved artifact at 'C:\Users\luise\AppData\Local\Temp\tmppf3rz10g'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 50, 6), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  2847604399040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2847604400976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2847605255056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2847605246432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2847605248720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2847614338848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2847614349936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2847614349232: TensorSpec(shape=(), dtype=tf.resource, name=None)
Model successfully exported to model.tflite


# Good job!
This is the end of Lab 3. In the next lab, you will focus on deploying your machine learning model onto your Android App in order to classify activities in real-time.

# GPT's suggested models

## model 1

In [57]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report

In [58]:
# Define parameters
input_shape = X_train.shape[1:]  # (window_size, num_features)
num_classes = y_train_one_hot.shape[1]  # 4 respiratory states

# Build the model
gpt_model1 = Sequential()

# Convolutional layers to capture spatial patterns
gpt_model1.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
gpt_model1.add(MaxPooling1D(pool_size=2))
gpt_model1.add(Dropout(0.3))

gpt_model1.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
gpt_model1.add(MaxPooling1D(pool_size=2))
gpt_model1.add(Dropout(0.3))

# LSTM layer to capture temporal dependencies
gpt_model1.add(LSTM(100, return_sequences=True))
gpt_model1.add(LSTM(50))

# Fully connected layers for classification
gpt_model1.add(Dense(50, activation='relu'))
gpt_model1.add(Dropout(0.5))
gpt_model1.add(Dense(num_classes, activation='softmax'))

# Compile the model
gpt_model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Set up callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5)
]

# Train the model
history = gpt_model1.fit(
    X_train, y_train_one_hot,
    epochs=50,
    batch_size=32,
    validation_data=(X_test, y_test_one_hot),
    callbacks=callbacks
)

Epoch 1/50
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 18s 9ms/step - accuracy: 0.4970 - loss: 1.2566 - val_accuracy: 0.5034 - val_loss: 1.2319 - learning_rate: 0.0010
Epoch 2/50
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - accuracy: 0.5107 - loss: 1.2213 - val_accuracy: 0.5719 - val_loss: 1.0060 - learning_rate: 0.0010
Epoch 3/50
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.5495 - loss: 1.0417 - val_accuracy: 0.5908 - val_loss: 0.9288 - learning_rate: 0.0010
Epoch 4/50
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.5634 - loss: 0.9765 - val_accuracy: 0.5919 - val_loss: 1.0029 - learning_rate: 0.0010
Epoch 5/50
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.5758 - loss: 0.9397 - val_accuracy: 0.6075 - val_loss: 0.9002 - learning_rate: 0.0010
Epoch 6/50
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - accuracy: 0.5796 - loss: 0.9260 - val_accuracy: 0.6044 - val_loss: 0.8637 - learning_rate: 0.0010
Epoch 7/50
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - accuracy: 0.5

### test

In [59]:
test(gpt_model1,X_test,y_test_one_hot)

449/449 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
              precision    recall  f1-score   support

           0     0.5484    0.5863    0.5667      2794
           1     0.7913    0.6593    0.7193      2163
           2     0.7555    0.2242    0.3458      2163
           3     0.6375    0.7868    0.7043      7217

    accuracy                         0.6436     14337
   macro avg     0.6832    0.5641    0.5840     14337
weighted avg     0.6612    0.6436    0.6257     14337

Per-class Recall:
Class 0: 0.5863
Class 1: 0.6593
Class 2: 0.2242
Class 3: 0.7868
Per-class Accuracy:
Class 0: 0.8253
Class 1: 0.9224
Class 2: 0.8720
Class 3: 0.6675


## model 2

In [54]:
from tensorflow.keras.layers import Bidirectional
from sklearn.utils import class_weight
import numpy as np

In [55]:
y_true_classes = np.argmax(y_train_one_hot, axis=1)
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_true_classes), y=y_true_classes)
class_weights_dict = dict(enumerate(class_weights))

In [60]:
# Update your model with deeper layers and bidirectional LSTM

gpt_model2 = Sequential()

# Convolutional layers
gpt_model2.add(Conv1D(filters=128, kernel_size=5, activation='relu', input_shape=input_shape))
gpt_model2.add(MaxPooling1D(pool_size=2))
gpt_model2.add(Dropout(0.4))

gpt_model2.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
gpt_model2.add(MaxPooling1D(pool_size=2))
gpt_model2.add(Dropout(0.4))

# Bidirectional LSTM layer
gpt_model2.add(Bidirectional(LSTM(150, return_sequences=True)))
gpt_model2.add(Bidirectional(LSTM(100)))

# Dense layers
gpt_model2.add(Dense(100, activation='relu'))
gpt_model2.add(Dropout(0.5))
gpt_model2.add(Dense(num_classes, activation='softmax'))

# Compile with a lower learning rate
gpt_model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train with class weights if needed
history = gpt_model2.fit(
    X_train, y_train_one_hot,
    epochs=100,
    batch_size=32,
    validation_data=(X_test, y_test_one_hot),
    callbacks=callbacks,
    class_weight=class_weights_dict  # Use class weights if classes are imbalanced
)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 26s 13ms/step - accuracy: 0.3064 - loss: 1.3275 - val_accuracy: 0.3158 - val_loss: 1.1178 - learning_rate: 5.0000e-04
Epoch 2/100
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 22s 12ms/step - accuracy: 0.3667 - loss: 1.0835 - val_accuracy: 0.3727 - val_loss: 1.0633 - learning_rate: 5.0000e-04
Epoch 3/100
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 22s 12ms/step - accuracy: 0.3875 - loss: 1.0026 - val_accuracy: 0.3857 - val_loss: 1.0716 - learning_rate: 5.0000e-04
Epoch 4/100
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 22s 12ms/step - accuracy: 0.4146 - loss: 0.9566 - val_accuracy: 0.3776 - val_loss: 1.1026 - learning_rate: 5.0000e-04
Epoch 5/100
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 22s 12ms/step - accuracy: 0.4288 - loss: 0.9323 - val_accuracy: 0.3995 - val_loss: 1.0757 - learning_rate: 5.0000e-04
Epoch 6/100
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 22s 12ms/step - accuracy: 0.4446 - loss: 0.9146 - val_accuracy: 0.4274 - val_loss: 1.0344 - learning_rate: 5.0000e-04
Epoch 7/100
1789/1789 ━━━━━━━━━━━━

### test

In [61]:
test(gpt_model2,X_test,y_test_one_hot)

449/449 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
              precision    recall  f1-score   support

           0     0.2554    0.9728    0.4045      2794
           1     0.5683    0.7443    0.6445      2163
           2     0.2310    0.0915    0.1311      2163
           3     0.3333    0.0001    0.0003      7217

    accuracy                         0.3158     14337
   macro avg     0.3470    0.4522    0.2951     14337
weighted avg     0.3382    0.3158    0.1960     14337

Per-class Recall:
Class 0: 0.9728
Class 1: 0.7443
Class 2: 0.0915
Class 3: 0.0001
Per-class Accuracy:
Class 0: 0.4419
Class 1: 0.8761
Class 2: 0.8170
Class 3: 0.4965


## best model with class balancing:

In [62]:
# Train with class weights if needed
history = best_model.fit(
    X_train, y_train_one_hot,
    epochs=100,
    batch_size=32,
    validation_data=(X_test, y_test_one_hot),
    callbacks=callbacks,
    class_weight=class_weights_dict  # Use class weights if classes are imbalanced
)

Epoch 1/100
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 17s 8ms/step - accuracy: 0.6131 - loss: 0.8182 - val_accuracy: 0.5888 - val_loss: 0.9240 - learning_rate: 0.0010
Epoch 2/100
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - accuracy: 0.6078 - loss: 0.8053 - val_accuracy: 0.5812 - val_loss: 0.9152 - learning_rate: 0.0010
Epoch 3/100
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.6036 - loss: 0.8006 - val_accuracy: 0.5821 - val_loss: 0.9478 - learning_rate: 0.0010
Epoch 4/100
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.6060 - loss: 0.7977 - val_accuracy: 0.5472 - val_loss: 0.9896 - learning_rate: 0.0010
Epoch 5/100
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.6113 - loss: 0.7893 - val_accuracy: 0.5958 - val_loss: 0.9317 - learning_rate: 0.0010
Epoch 6/100
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.6123 - loss: 0.7850 - val_accuracy: 0.6000 - val_loss: 0.9118 - learning_rate: 0.0010
Epoch 7/100
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accura

### test

In [63]:
test(best_model,X_test,y_test_one_hot)

449/449 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
              precision    recall  f1-score   support

           0     0.4429    0.9306    0.6001      2794
           1     0.6537    0.8141    0.7251      2163
           2     0.5534    0.6255    0.5872      2163
           3     0.8197    0.3779    0.5173      7217

    accuracy                         0.5888     14337
   macro avg     0.6174    0.6870    0.6074     14337
weighted avg     0.6810    0.5888    0.5753     14337

Per-class Recall:
Class 0: 0.9306
Class 1: 0.8141
Class 2: 0.6255
Class 3: 0.3779
Per-class Accuracy:
Class 0: 0.7583
Class 1: 0.9069
Class 2: 0.8673
Class 3: 0.6450


## adding summary features

In [64]:
import numpy as np
import scipy.stats as stats

In [65]:
def load_and_apply_sliding_windows2(file_paths, window_size, step_size, label):
    """
    Load the data from each file, apply sliding windows, and return the windows and labels.

    Parameters:
    file_paths (list): List of file paths to CSV files. Each file contains sensor data (e.g., accelerometer, gyroscope).
    window_size (int): The size of each sliding window (number of time steps).
    step_size (int): The step size (stride) between consecutive windows.
    label (int or str): The label for the activity corresponding to the folder.
                        This label will be assigned to each sliding window extracted from the data.

    Returns:
    tuple:
        - windows (numpy.ndarray): A 3D array of sliding windows, where each window has the shape
                                   (num_windows, window_size, num_features).
        - labels (numpy.ndarray): A 1D array of labels, where each label corresponds to a sliding window.
    """
    # Initialize lists to store sliding windows and their corresponding labels
    windows = []
    labels = []

    # Loop through each file in the provided file paths
    for file_path in file_paths:
        # Load the CSV file into a pandas DataFrame
        data = pd.read_csv(file_path)

        # Select the columns containing the necessary sensor data (acceleration and gyroscope readings)
        # These columns might vary depending on your dataset's structure
        data = data[['accel_x', 'accel_y', 'accel_z']]

        # Convert the DataFrame into a numpy array for faster processing in the sliding window operation
        data = data.to_numpy()

        # Get the number of samples (rows) and features (columns) in the data
        num_samples, num_features = data.shape

        # Apply sliding windows to the data
        # The range function defines the start of each window, moving step_size increments at a time
        for i in range(0, num_samples - window_size + 1, step_size):
            # Extract a window of size 'window_size' from the current position 'i'
            window = data[i:i + window_size, :]

            # Calculate statistics for each axis in the window
            mean = np.mean(window, axis=0)
            std = np.std(window, axis=0)
            min_val = np.min(window, axis=0)
            max_val = np.max(window, axis=0)
            skewness = stats.skew(window, axis=0)
            kurtosis = stats.kurtosis(window, axis=0)
            # Concatenate features into a single array
            window_features = np.concatenate([mean, std, min_val, max_val, skewness, kurtosis])

            # Reshape window_features to match the dimensions for concatenation
            # Expand dimensions to make it compatible with the window (1, num_statistical_features)
            window_features = np.expand_dims(window_features, axis=0)

            # Repeat window_features along the first dimension to match the window shape
            window_features_expanded = np.repeat(window_features, window.shape[0], axis=0)

            # Concatenate window_features with the original window along the last axis
            augmented_window = np.concatenate([window, window_features_expanded], axis=1)

            # Append the window to the windows list
            windows.append(augmented_window)

            # Assign the activity label to the window and append it to the labels list
            labels.append(label)


    # Convert the lists of windows and labels into numpy arrays for efficient numerical operations
    return np.array(windows), np.array(labels)

In [66]:
def process_activity2(activity, label, dataset_path, window_size=50, step_size=50, test_size=0.2):
    """
    Processes an activity folder by loading the file list, splitting them into
    train and test sets, and applying sliding windows to the files.

    Args:
        activity (str): Name of the activity (folder name). This refers to the specific physical activity
                        like 'walking', 'running', etc.
        label (int): Numeric label corresponding to the activity, used for classification.
        dataset_path (str): Base path where the activity folders are located.
        window_size (int): Size of the sliding window, i.e., the number of time steps included in each window.
                           Default is 50.
        step_size (int): Step size for the sliding window, i.e., how far the window moves along the data.
                         Default is 50 (no overlap between windows).
        test_size (float): Proportion of files to use for testing. Default is 0.2, meaning 20% of files will
                           be allocated to the test set.

    Returns:
        tuple:
            - train_windows (numpy.ndarray): Sliding windows from the training files.
            - train_labels (numpy.ndarray): Corresponding labels for the training windows.
            - test_windows (numpy.ndarray): Sliding windows from the test files.
            - test_labels (numpy.ndarray): Corresponding labels for the test windows.
    """
    # Construct the full folder path where the activity files are stored
    folder_path = os.path.join(dataset_path, activity)

    # Load all CSV file paths for the given activity from the folder
    file_list = load_files_from_folder(folder_path)

    # Split the file list into training and testing sets
    # train_files: files used for training
    # test_files: files used for testing
    train_files, test_files = split_files(file_list, test_size=test_size)

    # Apply sliding windows to the training files
    # The function 'load_and_apply_sliding_windows' returns the sliding windows (segments) and their corresponding labels
    train_windows, train_labels = load_and_apply_sliding_windows2(train_files, window_size, step_size, label)

    # Apply sliding windows to the testing files
    test_windows, test_labels = load_and_apply_sliding_windows2(test_files, window_size, step_size, label)

    # Return the sliding windows and their labels for both training and testing sets
    return train_windows, train_labels, test_windows, test_labels

In [67]:
# Dictionary to store sliding windows and labels for both train and test sets for each activity
# This will hold the training and test data after processing each activity.
train_test_data2 = {}

# Loop through each activity folder and process the data
# Note, if you have large amounts of data, this step may take a while
for activity, label in activities.items():
    # Initialize an empty dictionary for each activity to store train and test windows and labels
    train_test_data2[activity] = {}

    # Call process_activity() to process the data for the current activity folder
    # It loads the data, applies sliding windows, splits it into train and test sets,
    # and returns the respective sliding windows and labels for both sets.
    (train_test_data2[activity]['train_windows'], train_test_data2[activity]['train_labels'],
     train_test_data2[activity]['test_windows'], train_test_data2[activity]['test_labels']) = process_activity2(
        activity, label, your_dataset_path, window_size=50, step_size=50)

# Explanation:
    # - 'train_windows' and 'train_labels' store the windows and labels from the training files.
    # - 'test_windows' and 'test_labels' store the windows and labels from the test files.
    # - `your_dataset_path` should be replaced with the actual path to your dataset.
    # - `process_activity` handles all the steps of loading data, splitting it, and applying sliding windows.

In [68]:
# Combine the sliding windows and labels for the training data from all activities
# The combine_data() function concatenates the windows and labels across activities
X_train2, y_train2 = combine_data(train_test_data2, 'train')

# Combine the sliding windows and labels for the test data from all activities
X_test2, y_test2 = combine_data(train_test_data2, 'test')

# Explanation:
# - `combine_data()` takes in the `train_test_data` dictionary and the data type ('train' or 'test') to specify
#   whether we are combining training or testing data.
# - It retrieves and concatenates the windows and labels from all activities into single arrays
#   (`X_train` and `y_train` for training, `X_test` and `y_test` for testing).
# - `X_train` and `X_test` are 3D arrays of sliding windows (shape: num_windows, window_size, num_features).
# - `y_train` and `y_test` are 1D arrays containing the activity labels corresponding to each window.

In [69]:
# Print the shapes of the training and test arrays to verify that everything has been combined correctly
print(f"X_train2 shape: {X_train2.shape}, y_train2 shape: {y_train2.shape}")
print(f"X_test2 shape: {X_test2.shape}, y_test2 shape: {y_test2.shape}")

X_train2 shape: (57224, 50, 21), y_train2 shape: (57224,)
X_test2 shape: (14343, 50, 21), y_test2 shape: (14343,)


### model 3, whatever that is

In [70]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report

In [71]:
# Define the model architecture
model3 = Sequential()

# Convolutional layers
model3.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(50, 21)))
model3.add(MaxPooling1D(pool_size=2))
model3.add(Dropout(0.3))

model3.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model3.add(MaxPooling1D(pool_size=2))
model3.add(Dropout(0.3))

# LSTM layers for temporal dependencies
model3.add(LSTM(100, return_sequences=True))
model3.add(LSTM(50))

# Dense layers for classification
model3.add(Dense(50, activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(4, activation='softmax'))  # 4 classes

# Compile the model
model3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks for improved performance
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5)
]

# Train the model
history = model3.fit(
    X_train2, y_train2,
    epochs=50,
    batch_size=32,
    validation_data=(X_test2, y_test2),
    callbacks=callbacks
)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 18s 9ms/step - accuracy: 0.5412 - loss: 1.0697 - val_accuracy: 0.5896 - val_loss: 0.8707 - learning_rate: 0.0010
Epoch 2/50
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.5934 - loss: 0.8829 - val_accuracy: 0.5939 - val_loss: 0.8896 - learning_rate: 0.0010
Epoch 3/50
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.5991 - loss: 0.8624 - val_accuracy: 0.6055 - val_loss: 0.8660 - learning_rate: 0.0010
Epoch 4/50
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.6128 - loss: 0.8416 - val_accuracy: 0.5880 - val_loss: 0.8729 - learning_rate: 0.0010
Epoch 5/50
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.6155 - loss: 0.8271 - val_accuracy: 0.6112 - val_loss: 0.8580 - learning_rate: 0.0010
Epoch 6/50
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.6217 - loss: 0.8194 - val_accuracy: 0.6079 - val_loss: 0.8594 - learning_rate: 0.0010
Epoch 7/50
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.6

### test model 3

In [75]:
test(model3,X_test2,y_test_one_hot2)

449/449 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
              precision    recall  f1-score   support

           0     0.6821    0.4656    0.5534      2788
           1     0.5885    0.7368    0.6544      2166
           2     0.4980    0.2856    0.3630      2171
           3     0.6191    0.7276    0.6690      7218

    accuracy                         0.6112     14343
   macro avg     0.5969    0.5539    0.5599     14343
weighted avg     0.6084    0.6112    0.5980     14343

Per-class Recall:
Class 0: 0.4656
Class 1: 0.7368
Class 2: 0.2856
Class 3: 0.7276
Per-class Accuracy:
Class 0: 0.8539
Class 1: 0.8825
Class 2: 0.8483
Class 3: 0.6377


### using best model

using previous best model (not GPT's):

In [72]:
# Initialize the OneHotEncoder
encoder2 = OneHotEncoder(sparse_output=False)

# Reshape y_train to a 2D array to meet the input format requirements of OneHotEncoder
# - y_train is originally a 1D array of labels (shape: [num_samples]), but OneHotEncoder expects a 2D array of shape (num_samples, 1).
# - reshape(-1, 1): The -1 means 'infer the correct size based on the other dimensions' (i.e., it adapts based on the length of y_train).
# OneHotEncoder will then create a binary vector for each label.
y_train_one_hot2 = encoder2.fit_transform(y_train2.reshape(-1, 1))

# Apply the same transformation to the test labels (y_test)
# - Since the encoder is already fitted on the training data, we use transform() for the test set.
# - Reshape y_test to (num_samples, 1) for compatibility with the encoder.
y_test_one_hot2 = encoder2.transform(y_test2.reshape(-1, 1))

# Explanation:
# - y_train_one_hot and y_test_one_hot are now 2D arrays where each row is a one-hot encoded binary vector corresponding to a class label.
# - The number of columns in the one-hot encoded labels equals the number of unique classes (activities).
# For example, if there are 6 unique activities, the encoded vector will have 6 elements, with a '1' indicating the correct class.

In [73]:
# Print the shapes of the training and test arrays to verify that everything has been combined correctly
print(f"X_train2 shape: {X_train2.shape}, y_train2 shape: {y_train2.shape}")
print(f"X_test2 shape: {X_test2.shape}, y_test2 shape: {y_test2.shape}")

# Print the shapes of the one-hot encoded labels to verify that the transformation was successful
print(f"y_train_one_hot2 shape: {y_train_one_hot2.shape}, y_test_one_hot2 shape: {y_test_one_hot2.shape}")

# Explanation of shapes:
# - The shape of y_train_one_hot will be (num_samples, num_classes), where:
#     - num_samples is the number of training windows.
#     - num_classes is the number of unique activities (the length of the one-hot vectors).
# - Similarly, y_test_one_hot will have the same number of columns (num_classes) as y_train_one_hot but will have fewer rows (corresponding to the number of test windows).

X_train2 shape: (57224, 50, 21), y_train2 shape: (57224,)
X_test2 shape: (14343, 50, 21), y_test2 shape: (14343,)
y_train_one_hot2 shape: (57224, 4), y_test_one_hot2 shape: (14343, 4)


In [76]:
# Train the model
best_model2 = build_best_tuning_model(custom_input_shape=(50,21))
history = best_model2.fit(
    X_train2, y_train_one_hot2,
    epochs=50,
    batch_size=32,
    validation_data=(X_test2, y_test_one_hot2),
    callbacks=callbacks
)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_26 (Conv1D)                   │ (None, 46, 128)             │          13,568 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_14               │ (None, 46, 128)             │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_26 (MaxPooling1D)      │ (None, 23, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_35 (Dropout)                 │ (None, 23, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_27 (Conv1D)                   │ (None, 19, 256)             │         164,096 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_15               │ (None, 19, 256)             │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_27 (MaxPooling1D)      │ (None, 9, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_36 (Dropout)                 │ (None, 9, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_15 (LSTM)                       │ (None, 128)                 │         197,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_9 (Flatten)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_26 (Dense)                     │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_37 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_27 (Dense)                     │ (None, 4)                   │             516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 393,348 (1.50 MB)

 Trainable params: 392,580 (1.50 MB)

 Non-trainable params: 768 (3.00 KB)

Epoch 1/50
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 19s 9ms/step - accuracy: 0.5448 - loss: 1.1770 - val_accuracy: 0.5904 - val_loss: 0.9718 - learning_rate: 0.0010
Epoch 2/50
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - accuracy: 0.5836 - loss: 0.9685 - val_accuracy: 0.5809 - val_loss: 0.9616 - learning_rate: 0.0010
Epoch 3/50
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - accuracy: 0.5869 - loss: 0.9407 - val_accuracy: 0.5884 - val_loss: 0.9201 - learning_rate: 0.0010
Epoch 4/50
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - accuracy: 0.5938 - loss: 0.9305 - val_accuracy: 0.5950 - val_loss: 0.9098 - learning_rate: 0.0010
Epoch 5/50
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.5932 - loss: 0.9232 - val_accuracy: 0.5987 - val_loss: 0.9142 - learning_rate: 0.0010
Epoch 6/50
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.5984 - loss: 0.9135 - val_accuracy: 0.6126 - val_loss: 0.8996 - learning_rate: 0.0010
Epoch 7/50
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.5

### test best model

In [77]:
test(best_model2,X_test2,y_test_one_hot2)

449/449 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
              precision    recall  f1-score   support

           0     0.8360    0.2048    0.3290      2788
           1     0.6282    0.6357    0.6319      2166
           2     0.5496    0.0995    0.1685      2171
           3     0.5692    0.8734    0.6892      7218

    accuracy                         0.5904     14343
   macro avg     0.6458    0.4534    0.4547     14343
weighted avg     0.6270    0.5904    0.5317     14343

Per-class Recall:
Class 0: 0.2048
Class 1: 0.6357
Class 2: 0.0995
Class 3: 0.8734
Per-class Accuracy:
Class 0: 0.8376
Class 1: 0.8882
Class 2: 0.8514
Class 3: 0.6036


### Transformer model

In [78]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, MultiHeadAttention, LayerNormalization, Dropout, GlobalAveragePooling1D
from tensorflow.keras.models import Model

In [81]:
def build_transformer_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    # Multi-Head Attention Layer
    x = MultiHeadAttention(num_heads=4, key_dim=64)(inputs, inputs)
    x = LayerNormalization(epsilon=1e-6)(x)
    x = Dropout(0.3)(x)

    # Additional Feedforward Layer with residual connection
    x = Dense(128, activation='relu')(x)
    x = GlobalAveragePooling1D()(x)

    # Final dense layers for classification
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model


# Instantiate and train the Transformer model
transformer_model = build_transformer_model(input_shape=input_shape, num_classes=4)
history = transformer_model.fit(
    X_train, y_train,  # Note: y_train2 should be integer encoded, not one-hot for 'sparse_categorical_crossentropy'
    epochs=50,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5)
    ]
)

Epoch 1/50
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5048 - loss: 1.2481

ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 14337
'y' sizes: 84


### transformer test


# Models from papers

https://ieeexplore.ieee.org/abstract/document/10331170

gave up on this because their were layer shape mismatches in the model and it wasn't immediately obvious from the paper how they resolved those

also haven't implemented their input optimisation

In [82]:
def build_1d_cnn_model4(hp):
    """
    Builds and compiles a 1D CNN model for multi-class classification.

    Args:
        input_shape (tuple): The shape of the input data (timesteps, features).
        num_classes (int): The number of output classes.

    Returns:
        model (Sequential): Compiled 1D CNN model.
    """
    model = Sequential()

    # First Conv1D layer
    # You can try experimenting with different filters, kernel_size values and activiation functions
    model.add(Conv1D(filters=128, kernel_size=5, activation='relu', input_shape=input_shape))

    #model.add(MaxPooling1D(pool_size=2))

    # Second Conv1D layer
    # You can try experimenting with different filters, kernel_size values and activiation functions
    model.add(Conv1D(filters=128,
                     kernel_size=5,
                     activation='relu',
                     kernel_regularizer=l2(hp.Choice('l2_regularization', values=[1e-4, 1e-3])),
                     bias_regularizer=l2(hp.Choice('l2_regularization_bias', values=[1e-4, 1e-3]))
                    ))

    #model.add(MaxPooling1D(pool_size=2))

    # Flatten the output from the convolutional layers

    #model.add(Flatten())

    # Fully connected layer

    #model.add(Dense(128, activation='relu'))

    # Dropout layer for regularization
    # You can try experimenting with different dropout rates
    model.add(Dropout(0.3))

    model.add(MaxPooling1D(pool_size=2))

    model.add(Dense(100,
                    activation='softmax',
                     kernel_regularizer=l2(hp.Choice('l2_regularization', values=[1e-4, 1e-3])),
                     bias_regularizer=l2(hp.Choice('l2_regularization_bias', values=[1e-4, 1e-3]))
                    ))

    # Output layer with softmax for multi-class classification
    model.add(Dense(4,
                    activation='softmax',
                     kernel_regularizer=l2(hp.Choice('l2_regularization', values=[1e-4, 1e-3])),
                     bias_regularizer=l2(hp.Choice('l2_regularization_bias', values=[1e-4, 1e-3]))
                    ))

    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    #  Prints a detailed summary of the model, showing the layers, their output shapes, and the number of trainable parameters
    model.summary()

    return model

In [83]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, LSTM
from tensorflow.keras.regularizers import l2
import keras_tuner as kt

In [ ]:
tuner4 = kt.RandomSearch(
    build_1d_cnn_model4,
    objective='val_accuracy',  # Tuning for validation accuracy
    max_trials=20,             # Number of different hyperparameter sets to try
    executions_per_trial=2,    # Average results over multiple runs to reduce variance
    directory='my_tuning_dir', # Directory to save tuning results
    project_name='respiratory_activity_classification_model4'
)

# Split your data if not already done
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

tuner4.search(X_train, y_train_one_hot,
             epochs=20,               # Number of epochs for each trial
             validation_data=(X_test, y_test_one_hot),
             callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)])

model4 = tuner4.get_best_models(num_models=1)[0]

Reloading Tuner from my_tuning_dir\respiratory_activity_classification_model4\tuner0.json

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
0.001             |0.001             |l2_regularization
0.001             |0.001             |l2_regularization_bias



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 46, 128)        │         2,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 42, 128)        │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 42, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 21, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 21, 100)        │        12,900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 21, 4)          │           404 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 97,400 (380.47 KB)

 Trainable params: 97,400 (380.47 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20


Traceback (most recent call last):
  File "C:\Users\luise\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\luise\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\luise\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "C:\Users\luise\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-pack

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "C:\Users\luise\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\luise\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\luise\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "C:\Users\luise\AppData\Local\Packages\Py

window_size = 4s = 100 timesteps (untested)

In [ ]:
# Dictionary to store sliding windows and labels for both train and test sets for each activity
# This will hold the training and test data after processing each activity.
train_test_data3 = {}

# Loop through each activity folder and process the data
# Note, if you have large amounts of data, this step may take a while
for activity, label in activities.items():
    # Initialize an empty dictionary for each activity to store train and test windows and labels
    train_test_data3[activity] = {}

    # Call process_activity() to process the data for the current activity folder
    # It loads the data, applies sliding windows, splits it into train and test sets,
    # and returns the respective sliding windows and labels for both sets.
    (train_test_data3[activity]['train_windows'], train_test_data3[activity]['train_labels'],
     train_test_data3[activity]['test_windows'], train_test_data3[activity]['test_labels']) = process_activity(
        activity, label, your_dataset_path, window_size=100, step_size=50)

# Explanation:
    # - 'train_windows' and 'train_labels' store the windows and labels from the training files.
    # - 'test_windows' and 'test_labels' store the windows and labels from the test files.
    # - `your_dataset_path` should be replaced with the actual path to your dataset.
    # - `process_activity` handles all the steps of loading data, splitting it, and applying sliding windows.

In [ ]:
# Combine the sliding windows and labels for the training data from all activities
# The combine_data() function concatenates the windows and labels across activities
X_train3, y_train3 = combine_data(train_test_data3, 'train')

# Combine the sliding windows and labels for the test data from all activities
X_test3, y_test3 = combine_data(train_test_data3, 'test')

# Explanation:
# - `combine_data()` takes in the `train_test_data` dictionary and the data type ('train' or 'test') to specify
#   whether we are combining training or testing data.
# - It retrieves and concatenates the windows and labels from all activities into single arrays
#   (`X_train` and `y_train` for training, `X_test` and `y_test` for testing).
# - `X_train` and `X_test` are 3D arrays of sliding windows (shape: num_windows, window_size, num_features).
# - `y_train` and `y_test` are 1D arrays containing the activity labels corresponding to each window.

In [ ]:
# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Reshape y_train to a 2D array to meet the input format requirements of OneHotEncoder
# - y_train is originally a 1D array of labels (shape: [num_samples]), but OneHotEncoder expects a 2D array of shape (num_samples, 1).
# - reshape(-1, 1): The -1 means 'infer the correct size based on the other dimensions' (i.e., it adapts based on the length of y_train).
# OneHotEncoder will then create a binary vector for each label.
y_train_one_hot3 = encoder.fit_transform(y_train3.reshape(-1, 1))

# Apply the same transformation to the test labels (y_test)
# - Since the encoder is already fitted on the training data, we use transform() for the test set.
# - Reshape y_test to (num_samples, 1) for compatibility with the encoder.
y_test_one_hot3 = encoder.transform(y_test3.reshape(-1, 1))

# Explanation:
# - y_train_one_hot and y_test_one_hot are now 2D arrays where each row is a one-hot encoded binary vector corresponding to a class label.
# - The number of columns in the one-hot encoded labels equals the number of unique classes (activities).
# For example, if there are 6 unique activities, the encoded vector will have 6 elements, with a '1' indicating the correct class.

In [ ]:
# Print the shapes of the training and test arrays to verify that everything has been combined correctly
print(f"X_train3 shape: {X_train3.shape}, y_train3 shape: {y_train3.shape}")
print(f"X_test3 shape: {X_test3.shape}, y_test3 shape: {y_test3.shape}")
# Print the shapes of the one-hot encoded labels to verify that the transformation was successful
print(f"y_train_one_hot3 shape: {y_train_one_hot3.shape}, y_test_one_hot3 shape: {y_test_one_hot3.shape}")

# Explanation of shapes:
# - The shape of y_train_one_hot will be (num_samples, num_classes), where:
#     - num_samples is the number of training windows.
#     - num_classes is the number of unique activities (the length of the one-hot vectors).
# - Similarly, y_test_one_hot will have the same number of columns (num_classes) as y_train_one_hot but will have fewer rows (corresponding to the number of test windows).

## s-GAN
https://project-archive.inf.ed.ac.uk/ug4/20212442/ug4_proj.pdf#page=26.16

### data processing, window size 128 overlapping

In [34]:
# Dictionary to store sliding windows and labels for both train and test sets for each activity
# This will hold the training and test data after processing each activity.
train_test_data4 = {}

# Loop through each activity folder and process the data
# Note, if you have large amounts of data, this step may take a while
for activity, label in activities.items():
    # Initialize an empty dictionary for each activity to store train and test windows and labels
    train_test_data4[activity] = {}

    # Call process_activity() to process the data for the current activity folder
    # It loads the data, applies sliding windows, splits it into train and test sets,
    # and returns the respective sliding windows and labels for both sets.
    (train_test_data4[activity]['train_windows'], train_test_data4[activity]['train_labels'],
     train_test_data4[activity]['test_windows'], train_test_data4[activity]['test_labels']) = process_activity(
        activity, label, your_dataset_path, window_size=128, step_size=50)

# Explanation:
    # - 'train_windows' and 'train_labels' store the windows and labels from the training files.
    # - 'test_windows' and 'test_labels' store the windows and labels from the test files.
    # - `your_dataset_path` should be replaced with the actual path to your dataset.
    # - `process_activity` handles all the steps of loading data, splitting it, and applying sliding windows.
    # Combine the sliding windows and labels for the training data from all activities
# The combine_data() function concatenates the windows and labels across activities
X_train4, y_train4 = combine_data(train_test_data4, 'train')

# Combine the sliding windows and labels for the test data from all activities
X_test4, y_test4 = combine_data(train_test_data4, 'test')

# Explanation:
# - `combine_data()` takes in the `train_test_data` dictionary and the data type ('train' or 'test') to specify
#   whether we are combining training or testing data.
# - It retrieves and concatenates the windows and labels from all activities into single arrays
#   (`X_train` and `y_train` for training, `X_test` and `y_test` for testing).
# - `X_train` and `X_test` are 3D arrays of sliding windows (shape: num_windows, window_size, num_features).
# - `y_train` and `y_test` are 1D arrays containing the activity labels corresponding to each window.
# Initialize the OneHotEncoder
encoder4 = OneHotEncoder(sparse_output=False)

# Reshape y_train to a 2D array to meet the input format requirements of OneHotEncoder
# - y_train is originally a 1D array of labels (shape: [num_samples]), but OneHotEncoder expects a 2D array of shape (num_samples, 1).
# - reshape(-1, 1): The -1 means 'infer the correct size based on the other dimensions' (i.e., it adapts based on the length of y_train).
# OneHotEncoder will then create a binary vector for each label.
y_train_one_hot4 = encoder4.fit_transform(y_train4.reshape(-1, 1))

# Apply the same transformation to the test labels (y_test)
# - Since the encoder is already fitted on the training data, we use transform() for the test set.
# - Reshape y_test to (num_samples, 1) for compatibility with the encoder.
y_test_one_hot4 = encoder4.transform(y_test4.reshape(-1, 1))

# Explanation:
# - y_train_one_hot and y_test_one_hot are now 2D arrays where each row is a one-hot encoded binary vector corresponding to a class label.
# - The number of columns in the one-hot encoded labels equals the number of unique classes (activities).
# For example, if there are 6 unique activities, the encoded vector will have 6 elements, with a '1' indicating the correct class.
# Print the shapes of the training and test arrays to verify that everything has been combined correctly
print(f"X_train4 shape: {X_train4.shape}, y_train4 shape: {y_train4.shape}")
print(f"X_test4 shape: {X_test4.shape}, y_test4 shape: {y_test4.shape}")
# Print the shapes of the one-hot encoded labels to verify that the transformation was successful
print(f"y_train_one_hot4 shape: {y_train_one_hot4.shape}, y_test_one_hot4 shape: {y_test_one_hot4.shape}")

# Explanation of shapes:
# - The shape of y_train_one_hot will be (num_samples, num_classes), where:
#     - num_samples is the number of training windows.
#     - num_classes is the number of unique activities (the length of the one-hot vectors).
# - Similarly, y_test_one_hot will have the same number of columns (num_classes) as y_train_one_hot but will have fewer rows (corresponding to the number of test windows).

# Determine the input shape for the model
input_shape4 = (X_train4.shape[1], X_train4.shape[2])

X_train4 shape: (49898, 128, 3), y_train4 shape: (49898,)
X_test4 shape: (12500, 128, 3), y_test4 shape: (12500,)
y_train_one_hot4 shape: (49898, 4), y_test_one_hot4 shape: (12500, 4)


### model, window size 128

In [84]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

In [85]:
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, latent_dim=100, output_channels=3):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim

        self.fc1 = nn.Linear(latent_dim, 1024 * 4)  # Output: (batch_size, 4096)

        self.deconv1 = nn.Sequential(
            nn.ConvTranspose1d(1024, 512, kernel_size=4, stride=2, padding=1),  # Output: (batch_size, 512, 8)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5)
        )
        self.deconv2 = nn.Sequential(
            nn.ConvTranspose1d(512, 256, kernel_size=4, stride=2, padding=1),  # Output: (batch_size, 256, 16)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5)
        )
        self.deconv3 = nn.Sequential(
            nn.ConvTranspose1d(256, 128, kernel_size=4, stride=2, padding=1),  # Output: (batch_size, 128, 32)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5)
        )
        self.deconv4 = nn.Sequential(
            nn.ConvTranspose1d(128, 64, kernel_size=4, stride=2, padding=1),  # Output: (batch_size, 64, 64)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5)
        )
        self.deconv5 = nn.Sequential(
            nn.ConvTranspose1d(64, output_channels, kernel_size=4, stride=2, padding=1),  # Output: (batch_size, 3, 128)
            nn.Tanh()
        )

    def forward(self, z):
        x = self.fc1(z)
        x = x.view(-1, 1024, 4)
        x = self.deconv1(x)
        x = self.deconv2(x)
        x = self.deconv3(x)
        x = self.deconv4(x)
        x = self.deconv5(x)
        return x  # Output shape: (batch_size, 3, 128)

class Discriminator(nn.Module):
    def __init__(self, num_classes):
        super(Discriminator, self).__init__()
        self.num_classes = num_classes

        self.conv1 = nn.Sequential(
            nn.Conv1d(3, 512, kernel_size=4, stride=2, padding=1),  # Output: (batch_size, 512, 64)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5)
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(512, 1024, kernel_size=4, stride=2, padding=1),  # Output: (batch_size, 1024, 32)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5)
        )

        self.shared_features = nn.Flatten()  # Flattens to (batch_size, 1024*32)

        self.classifier = nn.Linear(1024 * 32, num_classes)
        self.discriminator = nn.Sequential(
            nn.Linear(1024 * 32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        features = self.shared_features(x)

        class_output = self.classifier(features)
        real_fake_output = self.discriminator(features)
        return class_output, real_fake_output


In [86]:
def train_sgan(generator, discriminator, dataloader, num_classes, num_epochs=30, latent_dim=100):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    generator.to(device)
    discriminator.to(device)

    criterion_class = nn.CrossEntropyLoss()
    criterion_real_fake = nn.BCELoss()

    optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))

    for epoch in range(num_epochs):
        for i, (x_real, labels_real) in enumerate(dataloader):
            x_real = x_real.to(device)  # Shape: (batch_size, 3, 128)
            labels_real = labels_real.to(device)
            batch_size = x_real.size(0)
            half_batch = batch_size // 2

            # Prepare labels
            real_labels = torch.ones(half_batch, 1).to(device)
            fake_labels = torch.zeros(half_batch, 1).to(device)

            # ====================
            # Train Discriminator
            # ====================
            discriminator.train()
            optimizer_D.zero_grad()

            # Train on real data
            x_real_half = x_real[:half_batch]
            labels_real_half = labels_real[:half_batch]
            class_output_real, real_fake_output_real = discriminator(x_real_half)
            d_loss_real_class = criterion_class(class_output_real, labels_real_half)
            d_loss_real_rf = criterion_real_fake(real_fake_output_real, real_labels)

            # Train on fake data
            z = torch.randn(half_batch, latent_dim).to(device)
            x_fake = generator(z)
            class_output_fake, real_fake_output_fake = discriminator(x_fake.detach())
            d_loss_fake_rf = criterion_real_fake(real_fake_output_fake, fake_labels)

            # Total discriminator loss
            d_loss = d_loss_real_class + d_loss_real_rf + d_loss_fake_rf
            d_loss.backward()
            optimizer_D.step()

            # ====================
            # Train Generator
            # ====================
            # Freeze discriminator parameters
            for param in discriminator.parameters():
                param.requires_grad = False

            generator.train()
            optimizer_G.zero_grad()

            # Generate fake data
            z = torch.randn(batch_size, latent_dim).to(device)
            x_fake = generator(z)

            # Forward pass through discriminator
            _, real_fake_output_fake = discriminator(x_fake)

            # Generator wants discriminator to classify fake data as real
            real_labels_g = torch.ones(batch_size, 1).to(device)
            g_loss_rf = criterion_real_fake(real_fake_output_fake, real_labels_g)
            g_loss = g_loss_rf
            g_loss.backward()
            optimizer_G.step()

            # Unfreeze discriminator parameters
            for param in discriminator.parameters():
                param.requires_grad = True

            if i % 100 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}] Batch [{i}/{len(dataloader)}] '
                      f'D_loss: {d_loss.item():.4f}, G_loss: {g_loss.item():.4f}')


In [87]:
import numpy as np

# Compute the mean and standard deviation from the training data
mean = X_train4.mean()
std = X_train4.std()

# Normalize the training and test data using the training mean and std
X_train4 = (X_train4 - mean) / std
X_test4 = (X_test4 - mean) / std

# Clip the data to the range [-1, 1]
X_train4 = np.clip(X_train4, -1, 1)
X_test4 = np.clip(X_test4, -1, 1)

In [88]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X  # NumPy array of shape (num_samples, seq_length, num_features)
        self.y = y  # NumPy array of shape (num_samples,)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        # Get the sample and label
        x = self.X[idx]  # Shape: (128, 3)
        y = self.y[idx]

        # Convert to torch tensors and transpose x to shape (num_features, seq_length)
        x = torch.tensor(x, dtype=torch.float32).transpose(0, 1)  # Shape: (3, 128)
        y = torch.tensor(y, dtype=torch.long)

        return x, y

In [89]:
batch_size = 128  # Adjust based on your memory constraints

# Create dataset and dataloader
train_dataset = CustomDataset(X_train4, y_train4)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Create dataset and dataloader
train_dataset = CustomDataset(X_train4, y_train4)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


In [37]:
# Example usage
if __name__ == '__main__':
    # Define hyperparameters
    latent_dim = 100
    num_classes = 4  # Replace with the actual number of activity classes
    num_epochs = 30
    batch_size = 128

    # Initialize models
    generator = Generator(latent_dim=latent_dim)
    discriminator = Discriminator(num_classes=num_classes)

    # Create DataLoader
    train_dataset = CustomDataset(X_train4, y_train4)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Train the S-GAN
    train_sgan(generator, discriminator, train_loader, num_classes, num_epochs, latent_dim)

Epoch [1/30] Batch [0/390] D_loss: 2.7945, G_loss: 0.6534
Epoch [1/30] Batch [100/390] D_loss: 1.4818, G_loss: 1.8593
Epoch [1/30] Batch [200/390] D_loss: 1.5227, G_loss: 8.2651
Epoch [1/30] Batch [300/390] D_loss: 1.7678, G_loss: 2.5286
Epoch [2/30] Batch [0/390] D_loss: 1.8856, G_loss: 1.6281
Epoch [2/30] Batch [100/390] D_loss: 1.9706, G_loss: 5.6919
Epoch [2/30] Batch [200/390] D_loss: 1.5695, G_loss: 2.2228
Epoch [2/30] Batch [300/390] D_loss: 1.7653, G_loss: 1.9664
Epoch [3/30] Batch [0/390] D_loss: 2.1089, G_loss: 1.9010
Epoch [3/30] Batch [100/390] D_loss: 1.7046, G_loss: 2.0794
Epoch [3/30] Batch [200/390] D_loss: 2.2641, G_loss: 1.4079
Epoch [3/30] Batch [300/390] D_loss: 2.2493, G_loss: 1.7989
Epoch [4/30] Batch [0/390] D_loss: 2.0315, G_loss: 1.6682
Epoch [4/30] Batch [100/390] D_loss: 1.8261, G_loss: 1.1798
Epoch [4/30] Batch [200/390] D_loss: 2.0601, G_loss: 1.7807
Epoch [4/30] Batch [300/390] D_loss: 2.1892, G_loss: 1.2485
Epoch [5/30] Batch [0/390] D_loss: 2.2203, G_los

### test

In [90]:
# Assuming you have already imported necessary libraries and defined CustomDataset

# Create test dataset
test_dataset = CustomDataset(X_test4, y_test4)

# Create DataLoader
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [91]:
# Set the discriminator to evaluation mode
discriminator.eval()


Discriminator(
  (conv1): Sequential(
    (0): Conv1d(3, 512, kernel_size=(4,), stride=(2,), padding=(1,))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Dropout(p=0.5, inplace=False)
  )
  (conv2): Sequential(
    (0): Conv1d(512, 1024, kernel_size=(4,), stride=(2,), padding=(1,))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Dropout(p=0.5, inplace=False)
  )
  (shared_features): Flatten(start_dim=1, end_dim=-1)
  (classifier): Linear(in_features=32768, out_features=4, bias=True)
  (discriminator): Sequential(
    (0): Linear(in_features=32768, out_features=1, bias=True)
    (1): Sigmoid()
  )
)

In [94]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Lists to store predictions and true labels
all_preds = []
all_labels = []
# Define the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with torch.no_grad():
    for x_test, y_test in test_loader:
        x_test = x_test.to(device)
        y_test = y_test.to(device)

        # Get the class outputs from the discriminator
        class_output, _ = discriminator(x_test)

        # Get the predicted class (the index with the highest score)
        _, preds = torch.max(class_output, 1)

        # Store predictions and true labels
        all_preds.append(preds.cpu().numpy())
        all_labels.append(y_test.cpu().numpy())

# Concatenate all predictions and labels into single arrays
all_preds = np.concatenate(all_preds)
all_labels = np.concatenate(all_labels)

# Compute overall accuracy
accuracy = accuracy_score(all_labels, all_preds)
print(f"Accuracy: {accuracy:.4f}")

# Generate a classification report
report = classification_report(all_labels, all_preds, digits=4)
print("Classification Report:")
print(report)

# Generate a confusion matrix
cm = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:")
print(cm)

# Compute per-class recall
classes = np.unique(all_labels)
per_class_recall = {}

for idx, cls in enumerate(classes):
    # True Positives (TP): Correct predictions for class 'cls'
    TP = cm[idx, idx]
    # False Negatives (FN): Actual class 'cls' but predicted differently
    FN = np.sum(cm[idx, :]) - TP

    # Recall = TP / (TP + FN)
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0.0
    per_class_recall[cls] = recall

# Print per-class recall
print("Per-class Recall:")
for cls, recall in per_class_recall.items():
    print(f"Class {cls}: {recall:.4f}")

per_class_accuracy = {}

for idx, cls in enumerate(classes):
    # True Positives (TP): Correct predictions for class 'cls'
    TP = cm[idx, idx]
    # True Negatives (TN): Correct predictions for all other classes
    TN = np.sum(cm) - (np.sum(cm[idx, :]) + np.sum(cm[:, idx]) - TP)
    # False Positives (FP): Incorrectly predicted as class 'cls'
    FP = np.sum(cm[:, idx]) - TP
    # False Negatives (FN): Actual class 'cls' but predicted differently
    FN = np.sum(cm[idx, :]) - TP

    # Per-class accuracy
    acc = (TP + TN) / total_samples
    per_class_accuracy[cls] = acc

# Print per-class accuracy
print("Per-class Accuracy:")
for cls, acc in per_class_accuracy.items():
    print(f"Class {cls}: {acc:.4f}")


Accuracy: 0.5550
Classification Report:
              precision    recall  f1-score   support

           0     0.7633    0.2357    0.3602      2435
           1     0.4764    0.5637    0.5164      1884
           2     0.3835    0.0986    0.1568      1887
           3     0.5663    0.8128    0.6675      6294

    accuracy                         0.5550     12500
   macro avg     0.5474    0.4277    0.4252     12500
weighted avg     0.5635    0.5550    0.5078     12500

Confusion Matrix:
[[ 574   25    7 1829]
 [  91 1062   68  663]
 [   6  269  186 1426]
 [  81  873  224 5116]]
Per-class Recall:
Class 0: 0.2357
Class 1: 0.5637
Class 2: 0.0986
Class 3: 0.8128
Per-class Accuracy:
Class 0: 0.8369
Class 1: 0.8409
Class 2: 0.8400
Class 3: 0.5923


## AC-GAN

### train

In [95]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [96]:
class Generator6(nn.Module):
  def __init__(self, latent_dim=100, num_classes=4, output_channels=3):
    super(Generator6, self).__init__()
    self.latent_dim = latent_dim
    self.num_classes = num_classes

    # Layers for processing noise vector z
    self.fc_z = nn.Linear(latent_dim, 1024 * 4)  # Output: (batch_size, 4096)

    # Layers for processing class labels c
    self.fc_c = nn.Linear(num_classes, 1024 * 4)  # Output: (batch_size, 4096)

    self.deconv1 = nn.Sequential(
      nn.ConvTranspose1d(2048, 512, kernel_size=4, stride=2, padding=1),  # Output: (batch_size, 512, 8)
      nn.LeakyReLU(0.2, inplace=True),
      nn.Dropout(0.5)
    )
    self.deconv2 = nn.Sequential(
      nn.ConvTranspose1d(512, 256, kernel_size=4, stride=2, padding=1),  # Output: (batch_size, 256, 16)
      nn.LeakyReLU(0.2, inplace=True),
      nn.Dropout(0.5)
    )
    self.deconv3 = nn.Sequential(
      nn.ConvTranspose1d(256, 128, kernel_size=4, stride=2, padding=1),  # Output: (batch_size, 128, 32)
      nn.LeakyReLU(0.2, inplace=True),
      nn.Dropout(0.5)
    )
    self.deconv4 = nn.Sequential(
      nn.ConvTranspose1d(128, 64, kernel_size=4, stride=2, padding=1),  # Output: (batch_size, 64, 64)
      nn.LeakyReLU(0.2, inplace=True),
      nn.Dropout(0.5)
    )
    self.deconv5 = nn.Sequential(
      nn.ConvTranspose1d(64, output_channels, kernel_size=4, stride=2, padding=1),  # Output: (batch_size, 3, 128)
      nn.Tanh()
    )

  def forward(self, z, c):
    # Process noise vector z
    x_z = self.fc_z(z)  # Shape: (batch_size, 4096)
    x_z = x_z.view(-1, 1024, 4)

    # One-hot encode class labels and process
    c = F.one_hot(c, num_classes=self.num_classes).float()
    x_c = self.fc_c(c)  # Shape: (batch_size, 4096)
    x_c = x_c.view(-1, 1024, 4)

    # Concatenate feature maps
    x = torch.cat([x_z, x_c], dim=1)  # Shape: (batch_size, 2048, 4)

    # Pass through deconvolutional layers
    x = self.deconv1(x)
    x = self.deconv2(x)
    x = self.deconv3(x)
    x = self.deconv4(x)
    x = self.deconv5(x)
    return x  # Output shape: (batch_size, 3, 128)


In [97]:
class Discriminator6(nn.Module):
    def __init__(self, num_classes):
        super(Discriminator6, self).__init__()
        self.num_classes = num_classes

        self.conv1 = nn.Sequential(
            nn.Conv1d(3, 512, kernel_size=4, stride=2, padding=1),  # Output: (batch_size, 512, 64)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5)
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(512, 1024, kernel_size=4, stride=2, padding=1),  # Output: (batch_size, 1024, 32)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5)
        )

        self.shared_features = nn.Flatten()  # Flattens to (batch_size, 1024*32)

        self.classifier = nn.Linear(1024 * 32, num_classes)
        self.discriminator = nn.Sequential(
            nn.Linear(1024 * 32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        features = self.shared_features(x)

        class_output = self.classifier(features)
        real_fake_output = self.discriminator(features)
        return class_output, real_fake_output

In [98]:
import numpy as np
from torch.utils.data import Dataset, DataLoader


def train_acgan6(generator, discriminator, dataloader, num_classes, num_epochs=30, latent_dim=100):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    generator.to(device)
    discriminator.to(device)

    criterion_class = nn.CrossEntropyLoss()
    criterion_real_fake = nn.BCELoss()

    optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))

    for epoch in range(num_epochs):
        for i, (x_real, labels_real) in enumerate(dataloader):
            x_real = x_real.to(device)
            labels_real = labels_real.to(device)
            batch_size = x_real.size(0)
            half_batch = batch_size // 2

            # Prepare labels
            real_labels = torch.ones(half_batch, 1).to(device)
            fake_labels = torch.zeros(half_batch, 1).to(device)

            # ====================
            # Train Discriminator
            # ====================
            discriminator.train()
            optimizer_D.zero_grad()

            # Train on real data
            x_real_half = x_real[:half_batch]
            labels_real_half = labels_real[:half_batch]
            class_output_real, real_fake_output_real = discriminator(x_real_half)
            d_loss_real_class = criterion_class(class_output_real, labels_real_half)
            d_loss_real_rf = criterion_real_fake(real_fake_output_real, real_labels)

            # Train on fake data
            z = torch.randn(half_batch, latent_dim).to(device)
            c_fake = torch.randint(0, num_classes, (half_batch,)).to(device)
            x_fake = generator(z, c_fake)
            class_output_fake, real_fake_output_fake = discriminator(x_fake.detach())
            d_loss_fake_class = criterion_class(class_output_fake, c_fake)
            d_loss_fake_rf = criterion_real_fake(real_fake_output_fake, fake_labels)

            # Total discriminator loss
            d_loss = d_loss_real_class + d_loss_real_rf + d_loss_fake_class + d_loss_fake_rf
            d_loss.backward()
            optimizer_D.step()

            # ====================
            # Train Generator
            # ====================
            # Freeze discriminator parameters
            for param in discriminator.parameters():
                param.requires_grad = False

            generator.train()
            optimizer_G.zero_grad()

            # Generate fake data
            z = torch.randn(batch_size, latent_dim).to(device)
            c_fake = torch.randint(0, num_classes, (batch_size,)).to(device)
            x_fake = generator(z, c_fake)

            # Forward pass through discriminator
            class_output_fake, real_fake_output_fake = discriminator(x_fake)

            # Generator wants discriminator to classify fake data as real and with correct class
            real_labels_g = torch.ones(batch_size, 1).to(device)
            g_loss_rf = criterion_real_fake(real_fake_output_fake, real_labels_g)
            g_loss_class = criterion_class(class_output_fake, c_fake)
            g_loss = g_loss_rf + g_loss_class
            g_loss.backward()
            optimizer_G.step()

            # Unfreeze discriminator parameters
            for param in discriminator.parameters():
                param.requires_grad = True

            if i % 100 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}] Batch [{i}/{len(dataloader)}] '
                      f'D_loss: {d_loss.item():.4f}, G_loss: {g_loss.item():.4f}')

# Normalize the data as before
mean6 = X_train4.mean()
std6 = X_train4.std()

X_train4 = (X_train4 - mean6) / std6
X_test4 = (X_test4 - mean6) / std6

X_train4 = np.clip(X_train4, -1, 1)
X_test4 = np.clip(X_test4, -1, 1)

class CustomDataset6(Dataset):
    def __init__(self, X, y):
        self.X = X  # NumPy array of shape (num_samples, seq_length, num_features)
        self.y = y  # NumPy array of shape (num_samples,)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        # Get the sample and label
        x = self.X[idx]  # Shape: (128, 3)
        y = self.y[idx]

        # Convert to torch tensors and transpose x to shape (num_features, seq_length)
        x = torch.tensor(x, dtype=torch.float32).transpose(0, 1)  # Shape: (3, 128)
        y = torch.tensor(y, dtype=torch.long)

        return x, y

In [99]:
# Example usage
from torch.utils.data import DataLoader, RandomSampler
if __name__ == '__main__':
    # Define hyperparameters
    latent_dim6 = 100
    num_classes6 = 4  # Replace with the actual number of activity classes

    #num_epochs6 = 30
    batch_size6 = 128

    # Initialize models
    generator6 = Generator6(latent_dim=latent_dim6, num_classes=num_classes6)
    discriminator6 = Discriminator6(num_classes=num_classes6)

    # Create DataLoader
    train_dataset6 = CustomDataset6(X_train4, y_train4)
    sampler6 = RandomSampler(train_dataset6, replacement=True)
    train_loader6 = DataLoader(train_dataset6, batch_size=batch_size6, sampler=sampler6)

    num_mini_batches_per_epoch = len(train_loader6)
    num_epochs6 = (25000 + num_mini_batches_per_epoch - 1) // num_mini_batches_per_epoch  # Ceiling division

    # Train the AC-GAN
    train_acgan6(generator6, discriminator6, train_loader6, num_classes6, num_epochs6, latent_dim6)


Epoch [1/65] Batch [0/390] D_loss: 4.1358, G_loss: 2.0260
Epoch [1/65] Batch [100/390] D_loss: 2.3976, G_loss: 3.4361
Epoch [1/65] Batch [200/390] D_loss: 1.3806, G_loss: 4.2746
Epoch [1/65] Batch [300/390] D_loss: 1.9244, G_loss: 3.3999
Epoch [2/65] Batch [0/390] D_loss: 1.5488, G_loss: 3.1709
Epoch [2/65] Batch [100/390] D_loss: 2.1223, G_loss: 3.6851
Epoch [2/65] Batch [200/390] D_loss: 2.2744, G_loss: 2.4735
Epoch [2/65] Batch [300/390] D_loss: 1.9704, G_loss: 1.3341
Epoch [3/65] Batch [0/390] D_loss: 2.3067, G_loss: 1.6056
Epoch [3/65] Batch [100/390] D_loss: 2.4991, G_loss: 2.5250
Epoch [3/65] Batch [200/390] D_loss: 2.0906, G_loss: 2.0461
Epoch [3/65] Batch [300/390] D_loss: 2.0035, G_loss: 2.0822
Epoch [4/65] Batch [0/390] D_loss: 1.8572, G_loss: 2.5043
Epoch [4/65] Batch [100/390] D_loss: 1.9339, G_loss: 2.6107
Epoch [4/65] Batch [200/390] D_loss: 2.1923, G_loss: 1.8695
Epoch [4/65] Batch [300/390] D_loss: 1.9960, G_loss: 1.9276
Epoch [5/65] Batch [0/390] D_loss: 2.1414, G_los

### test

In [100]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Assuming you have already imported necessary libraries and defined CustomDataset

# Create test dataset
test_dataset6 = CustomDataset6(X_test4, y_test4)

# Create DataLoader
test_loader6 = DataLoader(test_dataset6, batch_size=batch_size6, shuffle=False)

# Set the discriminator to evaluation mode
discriminator6.eval()

# Lists to store predictions and true labels
all_preds = []
all_labels = []
# Define the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with torch.no_grad():
    for x_test, y_test in test_loader:
        x_test = x_test.to(device)
        y_test = y_test.to(device)

        # Get the class outputs from the discriminator
        class_output, _ = discriminator(x_test)

        # Get the predicted class (the index with the highest score)
        _, preds = torch.max(class_output, 1)

        # Store predictions and true labels
        all_preds.append(preds.cpu().numpy())
        all_labels.append(y_test.cpu().numpy())

# Concatenate all predictions and labels into single arrays
all_preds = np.concatenate(all_preds)
all_labels = np.concatenate(all_labels)

# Compute overall accuracy
accuracy = accuracy_score(all_labels, all_preds)
print(f"Accuracy: {accuracy:.4f}")

# Generate a classification report
report = classification_report(all_labels, all_preds, digits=4)
print("Classification Report:")
print(report)

# Generate a confusion matrix
cm = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:")
print(cm)

# Compute per-class recall
classes = np.unique(all_labels)
per_class_recall = {}

for idx, cls in enumerate(classes):
    # True Positives (TP): Correct predictions for class 'cls'
    TP = cm[idx, idx]
    # False Negatives (FN): Actual class 'cls' but predicted differently
    FN = np.sum(cm[idx, :]) - TP

    # Recall = TP / (TP + FN)
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0.0
    per_class_recall[cls] = recall

# Print per-class recall
print("Per-class Recall:")
for cls, recall in per_class_recall.items():
    print(f"Class {cls}: {recall:.4f}")

per_class_accuracy = {}

for idx, cls in enumerate(classes):
    # True Positives (TP): Correct predictions for class 'cls'
    TP = cm[idx, idx]
    # True Negatives (TN): Correct predictions for all other classes
    TN = np.sum(cm) - (np.sum(cm[idx, :]) + np.sum(cm[:, idx]) - TP)
    # False Positives (FP): Incorrectly predicted as class 'cls'
    FP = np.sum(cm[:, idx]) - TP
    # False Negatives (FN): Actual class 'cls' but predicted differently
    FN = np.sum(cm[idx, :]) - TP

    # Per-class accuracy
    acc = (TP + TN) / total_samples
    per_class_accuracy[cls] = acc

# Print per-class accuracy
print("Per-class Accuracy:")
for cls, acc in per_class_accuracy.items():
    print(f"Class {cls}: {acc:.4f}")



Accuracy: 0.5550
Classification Report:
              precision    recall  f1-score   support

           0     0.7633    0.2357    0.3602      2435
           1     0.4764    0.5637    0.5164      1884
           2     0.3835    0.0986    0.1568      1887
           3     0.5663    0.8128    0.6675      6294

    accuracy                         0.5550     12500
   macro avg     0.5474    0.4277    0.4252     12500
weighted avg     0.5635    0.5550    0.5078     12500

Confusion Matrix:
[[ 574   25    7 1829]
 [  91 1062   68  663]
 [   6  269  186 1426]
 [  81  873  224 5116]]
Per-class Recall:
Class 0: 0.2357
Class 1: 0.5637
Class 2: 0.0986
Class 3: 0.8128
Per-class Accuracy:
Class 0: 0.8369
Class 1: 0.8409
Class 2: 0.8400
Class 3: 0.5923


### data processing, window size 24 non-overlapping

In [101]:
# Dictionary to store sliding windows and labels for both train and test sets for each activity
# This will hold the training and test data after processing each activity.
train_test_data24 = {}

# Loop through each activity folder and process the data
# Note, if you have large amounts of data, this step may take a while
for activity, label in activities.items():
    # Initialize an empty dictionary for each activity to store train and test windows and labels
    train_test_data24[activity] = {}

    # Call process_activity() to process the data for the current activity folder
    # It loads the data, applies sliding windows, splits it into train and test sets,
    # and returns the respective sliding windows and labels for both sets.
    (train_test_data24[activity]['train_windows'], train_test_data24[activity]['train_labels'],
     train_test_data24[activity]['test_windows'], train_test_data24[activity]['test_labels']) = process_activity(
        activity, label, your_dataset_path, window_size=24, step_size=24)

# Explanation:
    # - 'train_windows' and 'train_labels' store the windows and labels from the training files.
    # - 'test_windows' and 'test_labels' store the windows and labels from the test files.
    # - `your_dataset_path` should be replaced with the actual path to your dataset.
    # - `process_activity` handles all the steps of loading data, splitting it, and applying sliding windows.
    # Combine the sliding windows and labels for the training data from all activities
# The combine_data() function concatenates the windows and labels across activities
X_train24, y_train24 = combine_data(train_test_data24, 'train')

# Combine the sliding windows and labels for the test data from all activities
X_test24, y_test24 = combine_data(train_test_data24, 'test')

# Explanation:
# - `combine_data()` takes in the `train_test_data` dictionary and the data type ('train' or 'test') to specify
#   whether we are combining training or testing data.
# - It retrieves and concatenates the windows and labels from all activities into single arrays
#   (`X_train` and `y_train` for training, `X_test` and `y_test` for testing).
# - `X_train` and `X_test` are 3D arrays of sliding windows (shape: num_windows, window_size, num_features).
# - `y_train` and `y_test` are 1D arrays containing the activity labels corresponding to each window.
# Initialize the OneHotEncoder
encoder24 = OneHotEncoder(sparse_output=False)

# Reshape y_train to a 2D array to meet the input format requirements of OneHotEncoder
# - y_train is originally a 1D array of labels (shape: [num_samples]), but OneHotEncoder expects a 2D array of shape (num_samples, 1).
# - reshape(-1, 1): The -1 means 'infer the correct size based on the other dimensions' (i.e., it adapts based on the length of y_train).
# OneHotEncoder will then create a binary vector for each label.
y_train_one_hot24 = encoder24.fit_transform(y_train24.reshape(-1, 1))

# Apply the same transformation to the test labels (y_test)
# - Since the encoder is already fitted on the training data, we use transform() for the test set.
# - Reshape y_test to (num_samples, 1) for compatibility with the encoder.
y_test_one_hot24 = encoder24.transform(y_test24.reshape(-1, 1))

# Explanation:
# - y_train_one_hot and y_test_one_hot are now 2D arrays where each row is a one-hot encoded binary vector corresponding to a class label.
# - The number of columns in the one-hot encoded labels equals the number of unique classes (activities).
# For example, if there are 6 unique activities, the encoded vector will have 6 elements, with a '1' indicating the correct class.
# Print the shapes of the training and test arrays to verify that everything has been combined correctly
print(f"X_train24 shape: {X_train24.shape}, y_train24 shape: {y_train24.shape}")
print(f"X_test24 shape: {X_test24.shape}, y_test24 shape: {y_test24.shape}")
# Print the shapes of the one-hot encoded labels to verify that the transformation was successful
print(f"y_train_one_hot24 shape: {y_train_one_hot24.shape}, y_test_one_hot24 shape: {y_test_one_hot24.shape}")

# Explanation of shapes:
# - The shape of y_train_one_hot will be (num_samples, num_classes), where:
#     - num_samples is the number of training windows.
#     - num_classes is the number of unique activities (the length of the one-hot vectors).
# - Similarly, y_test_one_hot will have the same number of columns (num_classes) as y_train_one_hot but will have fewer rows (corresponding to the number of test windows).

# Determine the input shape for the model
input_shape24 = (X_train24.shape[1], X_train24.shape[2])

X_train24 shape: (118880, 24, 3), y_train24 shape: (118880,)
X_test24 shape: (29785, 24, 3), y_test24 shape: (29785,)
y_train_one_hot24 shape: (118880, 4), y_test_one_hot24 shape: (29785, 4)


### train, window size 24 non-overlapping

In [105]:
class Generator24(nn.Module):
    def __init__(self, latent_dim=100, num_classes=4, output_channels=3):
        super(Generator24, self).__init__()
        self.latent_dim = latent_dim
        self.num_classes = num_classes

        # Layers for processing noise vector z
        self.fc_z = nn.Linear(latent_dim, 1024 * 4)  # Output: (batch_size, 4096)

        # Layers for processing class labels c
        self.fc_c = nn.Linear(num_classes, 1024 * 4)  # Output: (batch_size, 4096)

        self.deconv1 = nn.Sequential(
            nn.ConvTranspose1d(2048, 512, kernel_size=3, stride=2, padding=1),  # Output: (batch_size, 512, 7)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5)
        )
        self.deconv2 = nn.Sequential(
            nn.ConvTranspose1d(512, 256, kernel_size=3, stride=2, padding=1),  # Output: (batch_size, 256, 13)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5)
        )
        self.deconv3 = nn.Sequential(
            nn.ConvTranspose1d(256, 128, kernel_size=4, stride=1, padding=1),  # Output: (batch_size, 128, 14)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5)
        )
        self.deconv4 = nn.Sequential(
            nn.ConvTranspose1d(128, 64, kernel_size=3, stride=1, padding=1),  # Output: (batch_size, 64, 14)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5)
        )
        self.deconv5 = nn.Sequential(
            nn.ConvTranspose1d(64, output_channels, kernel_size=11, stride=1, padding=0),  # Output: (batch_size, 3, 24)
            nn.Tanh()
        )

    def forward(self, z, c):
        # Process noise vector z
        x_z = self.fc_z(z)  # Shape: (batch_size, 4096)
        x_z = x_z.view(-1, 1024, 4)

        # One-hot encode class labels and process
        c = F.one_hot(c, num_classes=self.num_classes).float()
        x_c = self.fc_c(c)  # Shape: (batch_size, 4096)
        x_c = x_c.view(-1, 1024, 4)

        # Concatenate feature maps
        x = torch.cat([x_z, x_c], dim=1)  # Shape: (batch_size, 2048, 4)

        # Pass through deconvolutional layers
        x = self.deconv1(x)  # Output: (batch_size, 512, 7)
        x = self.deconv2(x)  # Output: (batch_size, 256, 13)
        x = self.deconv3(x)  # Output: (batch_size, 128, 14)
        x = self.deconv4(x)  # Output: (batch_size, 64, 14)
        x = self.deconv5(x)  # Output: (batch_size, 3, 24)
        return x  # Output shape: (batch_size, 3, 24)

In [109]:
class Discriminator24(nn.Module):
    def __init__(self, num_classes):
        super(Discriminator24, self).__init__()
        self.num_classes = num_classes

        self.conv1 = nn.Sequential(
            nn.Conv1d(3, 512, kernel_size=4, stride=2, padding=1),  # Output: (batch_size, 512, 12)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5)
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(512, 1024, kernel_size=4, stride=2, padding=1),  # Output: (batch_size, 1024, 6)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5)
        )

        self.shared_features = nn.Flatten()  # Flattens to (batch_size, 1024*6)

        self.classifier = nn.Linear(1024 * 6, num_classes)
        self.discriminator = nn.Sequential(
            nn.Linear(1024 * 6, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv1(x)  # Output: (batch_size, 512, 12)
        x = self.conv2(x)  # Output: (batch_size, 1024, 6)
        features = self.shared_features(x)

        class_output = self.classifier(features)
        real_fake_output = self.discriminator(features)
        return class_output, real_fake_output


In [110]:
# Example usage
from torch.utils.data import DataLoader, RandomSampler
if __name__ == '__main__':
    # Define hyperparameters
    latent_dim24 = 100
    num_classes24 = 4  # Replace with the actual number of activity classes

    #num_epochs6 = 30
    batch_size24 = 128

    # Initialize models
    generator24 = Generator24(latent_dim=latent_dim24, num_classes=num_classes24)
    discriminator24 = Discriminator24(num_classes=num_classes24)

    # Create DataLoader
    train_dataset24 = CustomDataset6(X_train24, y_train24)
    sampler24 = RandomSampler(train_dataset24, replacement=True)
    train_loader24 = DataLoader(train_dataset24, batch_size=batch_size24, sampler=sampler24)

    num_mini_batches_per_epoch = len(train_loader24)
    num_epochs24 = (25000 + num_mini_batches_per_epoch - 1) // num_mini_batches_per_epoch  # Ceiling division

    # Train the AC-GAN
    train_acgan6(generator24, discriminator24, train_loader24, num_classes24, num_epochs24, latent_dim24)


Epoch [1/27] Batch [0/929] D_loss: 4.1849, G_loss: 2.0776
Epoch [1/27] Batch [100/929] D_loss: 2.0287, G_loss: 1.8450
Epoch [1/27] Batch [200/929] D_loss: 2.4215, G_loss: 2.1883
Epoch [1/27] Batch [300/929] D_loss: 2.6351, G_loss: 2.7430
Epoch [1/27] Batch [400/929] D_loss: 2.5649, G_loss: 1.8901
Epoch [1/27] Batch [500/929] D_loss: 2.2857, G_loss: 2.1200
Epoch [1/27] Batch [600/929] D_loss: 2.4641, G_loss: 2.0242
Epoch [1/27] Batch [700/929] D_loss: 3.1856, G_loss: 1.9802
Epoch [1/27] Batch [800/929] D_loss: 2.5127, G_loss: 1.5335
Epoch [1/27] Batch [900/929] D_loss: 2.5783, G_loss: 1.7196
Epoch [2/27] Batch [0/929] D_loss: 2.5111, G_loss: 1.8425
Epoch [2/27] Batch [100/929] D_loss: 2.8252, G_loss: 1.6676
Epoch [2/27] Batch [200/929] D_loss: 2.6246, G_loss: 1.8482
Epoch [2/27] Batch [300/929] D_loss: 2.5394, G_loss: 1.7965
Epoch [2/27] Batch [400/929] D_loss: 2.4083, G_loss: 1.3879
Epoch [2/27] Batch [500/929] D_loss: 2.4138, G_loss: 1.5676
Epoch [2/27] Batch [600/929] D_loss: 2.4915,

### test

In [112]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Assuming you have already imported necessary libraries and defined CustomDataset

# Create test dataset
test_dataset24 = CustomDataset6(X_test24, y_test24)

# Create DataLoader
test_loader24 = DataLoader(test_dataset24, batch_size=batch_size24, shuffle=False)

# Set the discriminator to evaluation mode
discriminator24.eval()

def test_gan(test_loader, discriminator):

  # Lists to store predictions and true labels
  all_preds = []
  all_labels = []
  # Define the device
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  with torch.no_grad():
      for x_test, y_test in test_loader:
          x_test = x_test.to(device)
          y_test = y_test.to(device)

          # Get the class outputs from the discriminator
          class_output, _ = discriminator(x_test)

          # Get the predicted class (the index with the highest score)
          _, preds = torch.max(class_output, 1)

          # Store predictions and true labels
          all_preds.append(preds.cpu().numpy())
          all_labels.append(y_test.cpu().numpy())

  # Concatenate all predictions and labels into single arrays
  all_preds = np.concatenate(all_preds)
  all_labels = np.concatenate(all_labels)

  # Compute overall accuracy
  accuracy = accuracy_score(all_labels, all_preds)
  print(f"Accuracy: {accuracy:.4f}")

  # Generate a classification report
  report = classification_report(all_labels, all_preds, digits=4)
  print("Classification Report:")
  print(report)

  # Generate a confusion matrix
  cm = confusion_matrix(all_labels, all_preds)
  print("Confusion Matrix:")
  print(cm)

  # Compute per-class recall
  classes = np.unique(all_labels)
  per_class_recall = {}

  for idx, cls in enumerate(classes):
      # True Positives (TP): Correct predictions for class 'cls'
      TP = cm[idx, idx]
      # False Negatives (FN): Actual class 'cls' but predicted differently
      FN = np.sum(cm[idx, :]) - TP

      # Recall = TP / (TP + FN)
      recall = TP / (TP + FN) if (TP + FN) > 0 else 0.0
      per_class_recall[cls] = recall

  # Print per-class recall
  print("Per-class Recall:")
  for cls, recall in per_class_recall.items():
      print(f"Class {cls}: {recall:.4f}")

  per_class_accuracy = {}

  for idx, cls in enumerate(classes):
      # True Positives (TP): Correct predictions for class 'cls'
      TP = cm[idx, idx]
      # True Negatives (TN): Correct predictions for all other classes
      TN = np.sum(cm) - (np.sum(cm[idx, :]) + np.sum(cm[:, idx]) - TP)
      # False Positives (FP): Incorrectly predicted as class 'cls'
      FP = np.sum(cm[:, idx]) - TP
      # False Negatives (FN): Actual class 'cls' but predicted differently
      FN = np.sum(cm[idx, :]) - TP

      # Per-class accuracy
      acc = (TP + TN) / total_samples
      per_class_accuracy[cls] = acc

  # Print per-class accuracy
  print("Per-class Accuracy:")
  for cls, acc in per_class_accuracy.items():
      print(f"Class {cls}: {acc:.4f}")

test_gan(test_loader24, discriminator24)

Accuracy: 0.4940
Classification Report:
              precision    recall  f1-score   support

           0     0.4479    0.2815    0.3457      5805
           1     0.4525    0.5310    0.4886      4488
           2     0.2886    0.2186    0.2488      4510
           3     0.5563    0.6481    0.5987     14982

    accuracy                         0.4940     29785
   macro avg     0.4363    0.4198    0.4205     29785
weighted avg     0.4790    0.4940    0.4798     29785

Confusion Matrix:
[[1634  172  254 3745]
 [ 282 2383  390 1433]
 [ 278  679  986 2567]
 [1454 2032 1786 9710]]
Per-class Recall:
Class 0: 0.2815
Class 1: 0.5310
Class 2: 0.2186
Class 3: 0.6481
Per-class Accuracy:
Class 0: 1.8880
Class 1: 1.9838
Class 2: 1.9065
Class 3: 1.3414


### data processing, window size 24 overlapping

In [114]:
# Dictionary to store sliding windows and labels for both train and test sets for each activity
# This will hold the training and test data after processing each activity.
train_test_data24o = {}

# Loop through each activity folder and process the data
# Note, if you have large amounts of data, this step may take a while
for activity, label in activities.items():
    # Initialize an empty dictionary for each activity to store train and test windows and labels
    train_test_data24o[activity] = {}

    # Call process_activity() to process the data for the current activity folder
    # It loads the data, applies sliding windows, splits it into train and test sets,
    # and returns the respective sliding windows and labels for both sets.
    (train_test_data24o[activity]['train_windows'], train_test_data24o[activity]['train_labels'],
     train_test_data24o[activity]['test_windows'], train_test_data24o[activity]['test_labels']) = process_activity(
        activity, label, your_dataset_path, window_size=24, step_size=12)

# Explanation:
    # - 'train_windows' and 'train_labels' store the windows and labels from the training files.
    # - 'test_windows' and 'test_labels' store the windows and labels from the test files.
    # - `your_dataset_path` should be replaced with the actual path to your dataset.
    # - `process_activity` handles all the steps of loading data, splitting it, and applying sliding windows.
    # Combine the sliding windows and labels for the training data from all activities
# The combine_data() function concatenates the windows and labels across activities
X_train24o, y_train24o = combine_data(train_test_data24o, 'train')

# Combine the sliding windows and labels for the test data from all activities
X_test24o, y_test24o = combine_data(train_test_data24o, 'test')

# Explanation:
# - `combine_data()` takes in the `train_test_data` dictionary and the data type ('train' or 'test') to specify
#   whether we are combining training or testing data.
# - It retrieves and concatenates the windows and labels from all activities into single arrays
#   (`X_train` and `y_train` for training, `X_test` and `y_test` for testing).
# - `X_train` and `X_test` are 3D arrays of sliding windows (shape: num_windows, window_size, num_features).
# - `y_train` and `y_test` are 1D arrays containing the activity labels corresponding to each window.
# Initialize the OneHotEncoder
encoder24o = OneHotEncoder(sparse_output=False)

# Reshape y_train to a 2D array to meet the input format requirements of OneHotEncoder
# - y_train is originally a 1D array of labels (shape: [num_samples]), but OneHotEncoder expects a 2D array of shape (num_samples, 1).
# - reshape(-1, 1): The -1 means 'infer the correct size based on the other dimensions' (i.e., it adapts based on the length of y_train).
# OneHotEncoder will then create a binary vector for each label.
y_train_one_hot24o = encoder24o.fit_transform(y_train24o.reshape(-1, 1))

# Apply the same transformation to the test labels (y_test)
# - Since the encoder is already fitted on the training data, we use transform() for the test set.
# - Reshape y_test to (num_samples, 1) for compatibility with the encoder.
y_test_one_hot24o = encoder24o.transform(y_test24o.reshape(-1, 1))

# Explanation:
# - y_train_one_hot and y_test_one_hot are now 2D arrays where each row is a one-hot encoded binary vector corresponding to a class label.
# - The number of columns in the one-hot encoded labels equals the number of unique classes (activities).
# For example, if there are 6 unique activities, the encoded vector will have 6 elements, with a '1' indicating the correct class.
# Print the shapes of the training and test arrays to verify that everything has been combined correctly
print(f"X_train24 shape: {X_train24o.shape}, y_train24 shape: {y_train24o.shape}")
print(f"X_test24 shape: {X_test24o.shape}, y_test24 shape: {y_test24o.shape}")
# Print the shapes of the one-hot encoded labels to verify that the transformation was successful
print(f"y_train_one_hot24 shape: {y_train_one_hot24o.shape}, y_test_one_hot24 shape: {y_test_one_hot24o.shape}")

# Explanation of shapes:
# - The shape of y_train_one_hot will be (num_samples, num_classes), where:
#     - num_samples is the number of training windows.
#     - num_classes is the number of unique activities (the length of the one-hot vectors).
# - Similarly, y_test_one_hot will have the same number of columns (num_classes) as y_train_one_hot but will have fewer rows (corresponding to the number of test windows).

# Determine the input shape for the model
input_shape24o = (X_train24o.shape[1], X_train24o.shape[2])

X_train24 shape: (236924, 24, 3), y_train24 shape: (236924,)
X_test24 shape: (59379, 24, 3), y_test24 shape: (59379,)
y_train_one_hot24 shape: (236924, 4), y_test_one_hot24 shape: (59379, 4)


### train, window size 24 overlapping

In [117]:
# Example usage
from torch.utils.data import DataLoader, RandomSampler
if __name__ == '__main__':
    # Define hyperparameters
    latent_dim24o = 100
    num_classes24o = 4  # Replace with the actual number of activity classes

    #num_epochs6 = 30
    batch_size24o = 128

    # Initialize models
    generator24o = Generator24(latent_dim=latent_dim24o, num_classes=num_classes24o)
    discriminator24o = Discriminator24(num_classes=num_classes24o)

    # Create DataLoader
    train_dataset24o = CustomDataset6(X_train24o, y_train24o)
    sampler24o = RandomSampler(train_dataset24o, replacement=True)
    train_loader24o = DataLoader(train_dataset24o, batch_size=batch_size24o, sampler=sampler24o)

    num_mini_batches_per_epoch = len(train_loader24o)
    num_epochs24o = (25000 + num_mini_batches_per_epoch - 1) // num_mini_batches_per_epoch  # Ceiling division

    # Train the AC-GAN
    train_acgan6(generator24o, discriminator24o, train_loader24o, num_classes24o, num_epochs24o, latent_dim24o)


Epoch [1/14] Batch [0/1851] D_loss: 4.1794, G_loss: 2.1222
Epoch [1/14] Batch [100/1851] D_loss: 1.9169, G_loss: 2.0047
Epoch [1/14] Batch [200/1851] D_loss: 2.1982, G_loss: 2.6662
Epoch [1/14] Batch [300/1851] D_loss: 2.2979, G_loss: 2.1057
Epoch [1/14] Batch [400/1851] D_loss: 2.6313, G_loss: 2.3029
Epoch [1/14] Batch [500/1851] D_loss: 2.4399, G_loss: 1.9309
Epoch [1/14] Batch [600/1851] D_loss: 2.2821, G_loss: 1.8141
Epoch [1/14] Batch [700/1851] D_loss: 2.5884, G_loss: 1.5447
Epoch [1/14] Batch [800/1851] D_loss: 2.4811, G_loss: 1.8847
Epoch [1/14] Batch [900/1851] D_loss: 2.6047, G_loss: 1.6182
Epoch [1/14] Batch [1000/1851] D_loss: 2.8094, G_loss: 1.5535
Epoch [1/14] Batch [1100/1851] D_loss: 2.4821, G_loss: 1.6065
Epoch [1/14] Batch [1200/1851] D_loss: 2.6715, G_loss: 1.6061
Epoch [1/14] Batch [1300/1851] D_loss: 2.3117, G_loss: 1.5888
Epoch [1/14] Batch [1400/1851] D_loss: 2.6207, G_loss: 1.3975
Epoch [1/14] Batch [1500/1851] D_loss: 2.6623, G_loss: 1.4997
Epoch [1/14] Batch [

### test

In [118]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Assuming you have already imported necessary libraries and defined CustomDataset

# Create test dataset
test_dataset24o = CustomDataset6(X_test24o, y_test24o)

# Create DataLoader
test_loader24o = DataLoader(test_dataset24o, batch_size=batch_size24o, shuffle=False)

# Set the discriminator to evaluation mode
discriminator24o.eval()

test_gan(test_loader24o, discriminator24o)

Accuracy: 0.5196
Classification Report:
              precision    recall  f1-score   support

           0     0.5905    0.3124    0.4086     11572
           1     0.4955    0.4480    0.4705      8954
           2     0.2491    0.1375    0.1772      8975
           3     0.5470    0.7362    0.6276     29878

    accuracy                         0.5196     59379
   macro avg     0.4705    0.4085    0.4210     59379
weighted avg     0.5027    0.5196    0.4932     59379

Confusion Matrix:
[[ 3615   386   399  7172]
 [  268  4011   496  4179]
 [  193   685  1234  6863]
 [ 2046  3013  2824 21995]]
Per-class Recall:
Class 0: 0.3124
Class 1: 0.4480
Class 2: 0.1375
Class 3: 0.7362
Per-class Accuracy:
Class 0: 3.9132
Class 1: 4.0282
Class 2: 3.8335
Class 3: 2.6626


### data processing, window size 72 non-overlapping

In [119]:
# Dictionary to store sliding windows and labels for both train and test sets for each activity
# This will hold the training and test data after processing each activity.
train_test_data72 = {}

# Loop through each activity folder and process the data
# Note, if you have large amounts of data, this step may take a while
for activity, label in activities.items():
    # Initialize an empty dictionary for each activity to store train and test windows and labels
    train_test_data72[activity] = {}

    # Call process_activity() to process the data for the current activity folder
    # It loads the data, applies sliding windows, splits it into train and test sets,
    # and returns the respective sliding windows and labels for both sets.
    (train_test_data72[activity]['train_windows'], train_test_data72[activity]['train_labels'],
     train_test_data72[activity]['test_windows'], train_test_data72[activity]['test_labels']) = process_activity(
        activity, label, your_dataset_path, window_size=72, step_size=72)

# Explanation:
    # - 'train_windows' and 'train_labels' store the windows and labels from the training files.
    # - 'test_windows' and 'test_labels' store the windows and labels from the test files.
    # - `your_dataset_path` should be replaced with the actual path to your dataset.
    # - `process_activity` handles all the steps of loading data, splitting it, and applying sliding windows.
    # Combine the sliding windows and labels for the training data from all activities
# The combine_data() function concatenates the windows and labels across activities
X_train72, y_train72 = combine_data(train_test_data72, 'train')

# Combine the sliding windows and labels for the test data from all activities
X_test72, y_test72 = combine_data(train_test_data72, 'test')

# Explanation:
# - `combine_data()` takes in the `train_test_data` dictionary and the data type ('train' or 'test') to specify
#   whether we are combining training or testing data.
# - It retrieves and concatenates the windows and labels from all activities into single arrays
#   (`X_train` and `y_train` for training, `X_test` and `y_test` for testing).
# - `X_train` and `X_test` are 3D arrays of sliding windows (shape: num_windows, window_size, num_features).
# - `y_train` and `y_test` are 1D arrays containing the activity labels corresponding to each window.
# Initialize the OneHotEncoder
encoder72 = OneHotEncoder(sparse_output=False)

# Reshape y_train to a 2D array to meet the input format requirements of OneHotEncoder
# - y_train is originally a 1D array of labels (shape: [num_samples]), but OneHotEncoder expects a 2D array of shape (num_samples, 1).
# - reshape(-1, 1): The -1 means 'infer the correct size based on the other dimensions' (i.e., it adapts based on the length of y_train).
# OneHotEncoder will then create a binary vector for each label.
y_train_one_hot72 = encoder72.fit_transform(y_train72.reshape(-1, 1))

# Apply the same transformation to the test labels (y_test)
# - Since the encoder is already fitted on the training data, we use transform() for the test set.
# - Reshape y_test to (num_samples, 1) for compatibility with the encoder.
y_test_one_hot72 = encoder72.transform(y_test72.reshape(-1, 1))

# Explanation:
# - y_train_one_hot and y_test_one_hot are now 2D arrays where each row is a one-hot encoded binary vector corresponding to a class label.
# - The number of columns in the one-hot encoded labels equals the number of unique classes (activities).
# For example, if there are 6 unique activities, the encoded vector will have 6 elements, with a '1' indicating the correct class.
# Print the shapes of the training and test arrays to verify that everything has been combined correctly
print(f"X_train24 shape: {X_train72.shape}, y_train24 shape: {y_train72.shape}")
print(f"X_test24 shape: {X_test72.shape}, y_test24 shape: {y_test72.shape}")
# Print the shapes of the one-hot encoded labels to verify that the transformation was successful
print(f"y_train_one_hot24 shape: {y_train_one_hot72.shape}, y_test_one_hot24 shape: {y_test_one_hot72.shape}")

# Explanation of shapes:
# - The shape of y_train_one_hot will be (num_samples, num_classes), where:
#     - num_samples is the number of training windows.
#     - num_classes is the number of unique activities (the length of the one-hot vectors).
# - Similarly, y_test_one_hot will have the same number of columns (num_classes) as y_train_one_hot but will have fewer rows (corresponding to the number of test windows).

# Determine the input shape for the model
input_shape72 = (X_train72.shape[1], X_train72.shape[2])

X_train24 shape: (38299, 72, 3), y_train24 shape: (38299,)
X_test24 shape: (9600, 72, 3), y_test24 shape: (9600,)
y_train_one_hot24 shape: (38299, 4), y_test_one_hot24 shape: (9600, 4)


### train, window size 72 non-overlapping

In [120]:
class Generator72(nn.Module):
    def __init__(self, latent_dim=100, num_classes=4, output_channels=3):
        super(Generator72, self).__init__()
        self.latent_dim = latent_dim
        self.num_classes = num_classes

        # Layers for processing noise vector z
        self.fc_z = nn.Linear(latent_dim, 1024 * 4)  # Output: (batch_size, 4096)

        # Layers for processing class labels c
        self.fc_c = nn.Linear(num_classes, 1024 * 4)  # Output: (batch_size, 4096)

        self.deconv1 = nn.Sequential(
            nn.ConvTranspose1d(2048, 512, kernel_size=3, stride=2, padding=0),  # Output: (batch_size, 512, 9)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5)
        )
        self.deconv2 = nn.Sequential(
            nn.ConvTranspose1d(512, 256, kernel_size=4, stride=2, padding=1),  # Output: (batch_size, 256, 18)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5)
        )
        self.deconv3 = nn.Sequential(
            nn.ConvTranspose1d(256, 128, kernel_size=4, stride=2, padding=1),  # Output: (batch_size, 128, 36)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5)
        )
        self.deconv4 = nn.Sequential(
            nn.ConvTranspose1d(128, 64, kernel_size=4, stride=2, padding=1),  # Output: (batch_size, 64, 72)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5)
        )
        self.deconv5 = nn.Sequential(
            nn.ConvTranspose1d(64, output_channels, kernel_size=1, stride=1, padding=0),  # Output: (batch_size, 3, 72)
            nn.Tanh()
        )

    def forward(self, z, c):
        # Process noise vector z
        x_z = self.fc_z(z)  # Shape: (batch_size, 4096)
        x_z = x_z.view(-1, 1024, 4)

        # One-hot encode class labels and process
        c = F.one_hot(c, num_classes=self.num_classes).float()
        x_c = self.fc_c(c)  # Shape: (batch_size, 4096)
        x_c = x_c.view(-1, 1024, 4)

        # Concatenate feature maps
        x = torch.cat([x_z, x_c], dim=1)  # Shape: (batch_size, 2048, 4)

        # Pass through deconvolutional layers
        x = self.deconv1(x)  # Output: (batch_size, 512, 9)
        x = self.deconv2(x)  # Output: (batch_size, 256, 18)
        x = self.deconv3(x)  # Output: (batch_size, 128, 36)
        x = self.deconv4(x)  # Output: (batch_size, 64, 72)
        x = self.deconv5(x)  # Output: (batch_size, 3, 72)
        return x  # Output shape: (batch_size, 3, 72)

class Discriminator72(nn.Module):
    def __init__(self, num_classes):
        super(Discriminator72, self).__init__()
        self.num_classes = num_classes

        self.conv1 = nn.Sequential(
            nn.Conv1d(3, 512, kernel_size=4, stride=2, padding=1),  # Output: (batch_size, 512, 36)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5)
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(512, 1024, kernel_size=4, stride=2, padding=1),  # Output: (batch_size, 1024, 18)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5)
        )

        self.shared_features = nn.Flatten()  # Flattens to (batch_size, 1024*18)

        self.classifier = nn.Linear(1024 * 18, num_classes)
        self.discriminator = nn.Sequential(
            nn.Linear(1024 * 18, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv1(x)  # Output: (batch_size, 512, 36)
        x = self.conv2(x)  # Output: (batch_size, 1024, 18)
        features = self.shared_features(x)

        class_output = self.classifier(features)
        real_fake_output = self.discriminator(features)
        return class_output, real_fake_output


In [122]:
# Example usage
from torch.utils.data import DataLoader, RandomSampler
if __name__ == '__main__':
    # Define hyperparameters
    latent_dim72 = 100
    num_classes72 = 4  # Replace with the actual number of activity classes

    #num_epochs6 = 30
    batch_size72 = 128

    # Initialize models
    generator72 = Generator72(latent_dim=latent_dim72, num_classes=num_classes72)
    discriminator72 = Discriminator72(num_classes=num_classes72)

    # Create DataLoader
    train_dataset72 = CustomDataset6(X_train72, y_train72)
    sampler72 = RandomSampler(train_dataset72, replacement=True)
    train_loader72 = DataLoader(train_dataset72, batch_size=batch_size72, sampler=sampler72)

    num_mini_batches_per_epoch = len(train_loader72)
    num_epochs72 = (25000 + num_mini_batches_per_epoch - 1) // num_mini_batches_per_epoch  # Ceiling division

    # Train the AC-GAN
    train_acgan6(generator72, discriminator72, train_loader72, num_classes72, num_epochs72, latent_dim72)


Epoch [1/84] Batch [0/300] D_loss: 4.0881, G_loss: 2.1737
Epoch [1/84] Batch [100/300] D_loss: 2.2302, G_loss: 3.3645
Epoch [1/84] Batch [200/300] D_loss: 2.1454, G_loss: 3.1217
Epoch [2/84] Batch [0/300] D_loss: 2.3067, G_loss: 1.2715
Epoch [2/84] Batch [100/300] D_loss: 1.8568, G_loss: 2.5470
Epoch [2/84] Batch [200/300] D_loss: 1.9050, G_loss: 2.2484
Epoch [3/84] Batch [0/300] D_loss: 2.0873, G_loss: 4.6034
Epoch [3/84] Batch [100/300] D_loss: 1.8757, G_loss: 2.6112
Epoch [3/84] Batch [200/300] D_loss: 2.0928, G_loss: 1.4585
Epoch [4/84] Batch [0/300] D_loss: 1.9792, G_loss: 1.8203
Epoch [4/84] Batch [100/300] D_loss: 2.1962, G_loss: 1.4689
Epoch [4/84] Batch [200/300] D_loss: 2.2109, G_loss: 1.3459
Epoch [5/84] Batch [0/300] D_loss: 1.9818, G_loss: 2.0354
Epoch [5/84] Batch [100/300] D_loss: 2.6478, G_loss: 1.6014
Epoch [5/84] Batch [200/300] D_loss: 2.1687, G_loss: 2.1877
Epoch [6/84] Batch [0/300] D_loss: 2.0938, G_loss: 1.8615
Epoch [6/84] Batch [100/300] D_loss: 1.7965, G_loss:

### test

In [123]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Assuming you have already imported necessary libraries and defined CustomDataset

# Create test dataset
test_dataset72 = CustomDataset6(X_test72, y_test72)

# Create DataLoader
test_loader72 = DataLoader(test_dataset72, batch_size=batch_size72, shuffle=False)

# Set the discriminator to evaluation mode
discriminator72.eval()


test_gan(test_loader72, discriminator72)


Accuracy: 0.4677
Classification Report:
              precision    recall  f1-score   support

           0     0.5418    0.4984    0.5192      1872
           1     0.4785    0.3460    0.4016      1445
           2     0.2355    0.4415    0.3072      1454
           3     0.5880    0.5001    0.5405      4829

    accuracy                         0.4677      9600
   macro avg     0.4610    0.4465    0.4421      9600
weighted avg     0.5091    0.4677    0.4801      9600

Confusion Matrix:
[[ 933   17  264  658]
 [  15  500  526  404]
 [ 104   78  642  630]
 [ 670  450 1294 2415]]
Per-class Recall:
Class 0: 0.4984
Class 1: 0.3460
Class 2: 0.4415
Class 3: 0.5001
Per-class Accuracy:
Class 0: 0.6298
Class 1: 0.6488
Class 2: 0.5363
Class 3: 0.4395


### data processing, window size 72 overlapping

In [124]:
# Dictionary to store sliding windows and labels for both train and test sets for each activity
# This will hold the training and test data after processing each activity.
train_test_data72o = {}

# Loop through each activity folder and process the data
# Note, if you have large amounts of data, this step may take a while
for activity, label in activities.items():
    # Initialize an empty dictionary for each activity to store train and test windows and labels
    train_test_data72o[activity] = {}

    # Call process_activity() to process the data for the current activity folder
    # It loads the data, applies sliding windows, splits it into train and test sets,
    # and returns the respective sliding windows and labels for both sets.
    (train_test_data72o[activity]['train_windows'], train_test_data72o[activity]['train_labels'],
     train_test_data72o[activity]['test_windows'], train_test_data72o[activity]['test_labels']) = process_activity(
        activity, label, your_dataset_path, window_size=72, step_size=36)

# Explanation:
    # - 'train_windows' and 'train_labels' store the windows and labels from the training files.
    # - 'test_windows' and 'test_labels' store the windows and labels from the test files.
    # - `your_dataset_path` should be replaced with the actual path to your dataset.
    # - `process_activity` handles all the steps of loading data, splitting it, and applying sliding windows.
    # Combine the sliding windows and labels for the training data from all activities
# The combine_data() function concatenates the windows and labels across activities
X_train72o, y_train72o = combine_data(train_test_data72o, 'train')

# Combine the sliding windows and labels for the test data from all activities
X_test72o, y_test72o = combine_data(train_test_data72o, 'test')

# Explanation:
# - `combine_data()` takes in the `train_test_data` dictionary and the data type ('train' or 'test') to specify
#   whether we are combining training or testing data.
# - It retrieves and concatenates the windows and labels from all activities into single arrays
#   (`X_train` and `y_train` for training, `X_test` and `y_test` for testing).
# - `X_train` and `X_test` are 3D arrays of sliding windows (shape: num_windows, window_size, num_features).
# - `y_train` and `y_test` are 1D arrays containing the activity labels corresponding to each window.
# Initialize the OneHotEncoder
encoder72o = OneHotEncoder(sparse_output=False)

# Reshape y_train to a 2D array to meet the input format requirements of OneHotEncoder
# - y_train is originally a 1D array of labels (shape: [num_samples]), but OneHotEncoder expects a 2D array of shape (num_samples, 1).
# - reshape(-1, 1): The -1 means 'infer the correct size based on the other dimensions' (i.e., it adapts based on the length of y_train).
# OneHotEncoder will then create a binary vector for each label.
y_train_one_hot72o = encoder72o.fit_transform(y_train72o.reshape(-1, 1))

# Apply the same transformation to the test labels (y_test)
# - Since the encoder is already fitted on the training data, we use transform() for the test set.
# - Reshape y_test to (num_samples, 1) for compatibility with the encoder.
y_test_one_hot72o = encoder72o.transform(y_test72o.reshape(-1, 1))

# Explanation:
# - y_train_one_hot and y_test_one_hot are now 2D arrays where each row is a one-hot encoded binary vector corresponding to a class label.
# - The number of columns in the one-hot encoded labels equals the number of unique classes (activities).
# For example, if there are 6 unique activities, the encoded vector will have 6 elements, with a '1' indicating the correct class.
# Print the shapes of the training and test arrays to verify that everything has been combined correctly
print(f"X_train24 shape: {X_train72o.shape}, y_train24 shape: {y_train72o.shape}")
print(f"X_test24 shape: {X_test72o.shape}, y_test24 shape: {y_test72o.shape}")
# Print the shapes of the one-hot encoded labels to verify that the transformation was successful
print(f"y_train_one_hot24 shape: {y_train_one_hot72o.shape}, y_test_one_hot24 shape: {y_test_one_hot72o.shape}")

# Explanation of shapes:
# - The shape of y_train_one_hot will be (num_samples, num_classes), where:
#     - num_samples is the number of training windows.
#     - num_classes is the number of unique activities (the length of the one-hot vectors).
# - Similarly, y_test_one_hot will have the same number of columns (num_classes) as y_train_one_hot but will have fewer rows (corresponding to the number of test windows).

# Determine the input shape for the model
input_shape72o = (X_train72o.shape[1], X_train72o.shape[2])

X_train24 shape: (75892, 72, 3), y_train24 shape: (75892,)
X_test24 shape: (18987, 72, 3), y_test24 shape: (18987,)
y_train_one_hot24 shape: (75892, 4), y_test_one_hot24 shape: (18987, 4)


### train, window size 72 overlapping

In [125]:
# Example usage
from torch.utils.data import DataLoader, RandomSampler
if __name__ == '__main__':
    # Define hyperparameters
    latent_dim72o = 100
    num_classes72o = 4  # Replace with the actual number of activity classes

    #num_epochs6 = 30
    batch_size72o = 128

    # Initialize models
    generator72o = Generator72(latent_dim=latent_dim72o, num_classes=num_classes72o)
    discriminator72o = Discriminator72(num_classes=num_classes72o)

    # Create DataLoader
    train_dataset72o = CustomDataset6(X_train72o, y_train72o)
    sampler72o = RandomSampler(train_dataset72o, replacement=True)
    train_loader72o = DataLoader(train_dataset72o, batch_size=batch_size72o, sampler=sampler72o)

    num_mini_batches_per_epoch = len(train_loader72o)
    num_epochs72o = (25000 + num_mini_batches_per_epoch - 1) // num_mini_batches_per_epoch  # Ceiling division

    # Train the AC-GAN
    train_acgan6(generator72o, discriminator72o, train_loader72o, num_classes72o, num_epochs72o, latent_dim72o)


Epoch [1/43] Batch [0/593] D_loss: 4.1759, G_loss: 2.2217
Epoch [1/43] Batch [100/593] D_loss: 1.9182, G_loss: 2.7678
Epoch [1/43] Batch [200/593] D_loss: 2.0331, G_loss: 2.6763
Epoch [1/43] Batch [300/593] D_loss: 1.7734, G_loss: 1.6039
Epoch [1/43] Batch [400/593] D_loss: 1.8406, G_loss: 2.8029
Epoch [1/43] Batch [500/593] D_loss: 1.8020, G_loss: 2.6222
Epoch [2/43] Batch [0/593] D_loss: 1.5855, G_loss: 3.4269
Epoch [2/43] Batch [100/593] D_loss: 1.7301, G_loss: 2.5392
Epoch [2/43] Batch [200/593] D_loss: 1.7701, G_loss: 2.2627
Epoch [2/43] Batch [300/593] D_loss: 1.8602, G_loss: 3.0396
Epoch [2/43] Batch [400/593] D_loss: 1.8051, G_loss: 2.6668
Epoch [2/43] Batch [500/593] D_loss: 1.9748, G_loss: 2.2147
Epoch [3/43] Batch [0/593] D_loss: 2.0992, G_loss: 1.8601
Epoch [3/43] Batch [100/593] D_loss: 2.2935, G_loss: 2.4408
Epoch [3/43] Batch [200/593] D_loss: 3.0887, G_loss: 2.0907
Epoch [3/43] Batch [300/593] D_loss: 2.0157, G_loss: 2.0255
Epoch [3/43] Batch [400/593] D_loss: 2.3830, G

### test

In [126]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Assuming you have already imported necessary libraries and defined CustomDataset

# Create test dataset
test_dataset72o = CustomDataset6(X_test72o, y_test72o)

# Create DataLoader
test_loader72o = DataLoader(test_dataset72o, batch_size=batch_size72o, shuffle=False)

# Set the discriminator to evaluation mode
discriminator72o.eval()

test_gan(test_loader72o, discriminator72o)

Accuracy: 0.5542
Classification Report:
              precision    recall  f1-score   support

           0     0.5489    0.5935    0.5704      3695
           1     0.4964    0.6474    0.5619      2870
           2     0.4359    0.1971    0.2714      2862
           3     0.5935    0.6180    0.6055      9560

    accuracy                         0.5542     18987
   macro avg     0.5187    0.5140    0.5023     18987
weighted avg     0.5464    0.5542    0.5417     18987

Confusion Matrix:
[[2193   98    5 1399]
 [  29 1858  167  816]
 [  93  373  564 1832]
 [1680 1414  558 5908]]
Per-class Recall:
Class 0: 0.5935
Class 1: 0.6474
Class 2: 0.1971
Class 3: 0.6180
Per-class Accuracy:
Class 0: 1.2546
Class 1: 1.2872
Class 2: 1.2767
Class 3: 0.9030


## ResNet

https://project-archive.inf.ed.ac.uk/ug4/20212442/ug4_proj.pdf#page=26.16

In [73]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, Dropout, GlobalMaxPooling2D, Add, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [74]:
# Define input shape and number of classes
input_shape7 = (64, 64, 3)  # Example input shape
num_classes7 = 4  # Number of activities

# Input layer
inputs7 = Input(shape=input_shape7)

# Branch 1: Downsample Conv (128 filters)
downsample7 = Conv2D(128, kernel_size=(1,1), strides=(1,1), padding='same')(inputs7)

# Branch 2: Convolutional Block
# Conv1 (128 filters) + BatchNorm + ReLU
x = Conv2D(128, kernel_size=(3,3), padding='same')(inputs7)
x = BatchNormalization()(x)
x = ReLU()(x)

# Conv2 (128 filters) + BatchNorm + ReLU
x = Conv2D(128, kernel_size=(3,3), padding='same')(x)
x = BatchNormalization()(x)
x = ReLU()(x)

# Conv3 (128 filters) + Dropout + BatchNorm
x = Conv2D(128, kernel_size=(3,3), padding='same')(x)
x = Dropout(0.5)(x)  # 0.5 probability of keeping each neuron
x = BatchNormalization()(x)

# Residual Connection
x = Add()([downsample7, x])

# Activation and Dropout after addition
x = ReLU()(x)
x = Dropout(0.5)(x)

# Global Max Pooling
x = GlobalMaxPooling2D()(x)

# Output layer with 'n' units and softmax activation
outputs7 = Dense(num_classes7, activation='softmax')(x)

# Create the model
model7 = Model(inputs=inputs7, outputs=outputs7)

# Compile the model with categorical cross-entropy loss and Adam optimizer
model7.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Define EarlyStopping callback with patience of 25 epochs
early_stopping7 = EarlyStopping(patience=25, restore_best_weights=True)

In [ ]:
# Assuming train_data, train_labels, val_data, val_labels are predefined datasets
# Train the model for 50 epochs with batch size of 128
history = model.fit(
    train_data, train_labels,
    validation_data=(val_data, val_labels),
    epochs=50,
    batch_size=128,
    callbacks=[early_stopping]
)
